***
# ***trans*-Azo system** <a name="Phe_system"></a>
***

### **Content:**
&emsp;[1. Setup](#setup)  
&emsp;[2. Trajectory preprocessing](#trajectory_preprocessing)   
&emsp;[3. Trajectory visualization](#video)   
&emsp;[4. Calculation](#calculation)   
&emsp;[5. Thermodynamics](#thermodynamics)    
&emsp;[6. Water distribution density](#water)    
&emsp;[6. Trypticene stand tilt](#lower_tilt)   
&emsp;[8. Functional head tilt](#upper_tilt)   
&emsp;[9. Functional head bend](#bend)   
&emsp;[10. Upper layer interactions](#upper_layer)   
&emsp;[11. Lower layer interactions](#lower_layer)   
&emsp;[12. Combined distribution functions](#cdf)   

<a name="setup"></a>
## **Setup**
***

In [ ]:
%%js
/** vim: et:ts=4:sw=4:sts=4
 * @license RequireJS 2.3.6 Copyright jQuery Foundation and other contributors.
* Released under MIT license, https://github.com/requirejs/requirejs/blob/master/LICENSE
 */
//Not using strict: uneven strict support in browsers, #392, and causes
//problems with requirejs.exec()/transpiler plugins that may not be strict.
/*jslint regexp: true, nomen: true, sloppy: true */
/*global window, navigator, document, importScripts, setTimeout, opera */

var requirejs, require, define;
(function (global, setTimeout) {
    var req, s, head, baseElement, dataMain, src,
        interactiveScript, currentlyAddingScript, mainScript, subPath,
        version = '2.3.6',
        commentRegExp = /\/\*[\s\S]*?\*\/|([^:"'=]|^)\/\/.*$/mg,
        cjsRequireRegExp = /[^.]\s*require\s*\(\s*["']([^'"\s]+)["']\s*\)/g,
        jsSuffixRegExp = /\.js$/,
        currDirRegExp = /^\.\//,
        op = Object.prototype,
        ostring = op.toString,
        hasOwn = op.hasOwnProperty,
        isBrowser = !!(typeof window !== 'undefined' && typeof navigator !== 'undefined' && window.document),
        isWebWorker = !isBrowser && typeof importScripts !== 'undefined',
        //PS3 indicates loaded and complete, but need to wait for complete
        //specifically. Sequence is 'loading', 'loaded', execution,
        // then 'complete'. The UA check is unfortunate, but not sure how
        //to feature test w/o causing perf issues.
        readyRegExp = isBrowser && navigator.platform === 'PLAYSTATION 3' ?
                      /^complete$/ : /^(complete|loaded)$/,
        defContextName = '_',
        //Oh the tragedy, detecting opera. See the usage of isOpera for reason.
        isOpera = typeof opera !== 'undefined' && opera.toString() === '[object Opera]',
        contexts = {},
        cfg = {},
        globalDefQueue = [],
        useInteractive = false;

    //Could match something like ')//comment', do not lose the prefix to comment.
    function commentReplace(match, singlePrefix) {
        return singlePrefix || '';
    }

    function isFunction(it) {
        return ostring.call(it) === '[object Function]';
    }

    function isArray(it) {
        return ostring.call(it) === '[object Array]';
    }

    /**
     * Helper function for iterating over an array. If the func returns
     * a true value, it will break out of the loop.
     */
    function each(ary, func) {
        if (ary) {
            var i;
            for (i = 0; i < ary.length; i += 1) {
                if (ary[i] && func(ary[i], i, ary)) {
                    break;
                }
            }
        }
    }

    /**
     * Helper function for iterating over an array backwards. If the func
     * returns a true value, it will break out of the loop.
     */
    function eachReverse(ary, func) {
        if (ary) {
            var i;
            for (i = ary.length - 1; i > -1; i -= 1) {
                if (ary[i] && func(ary[i], i, ary)) {
                    break;
                }
            }
        }
    }

    function hasProp(obj, prop) {
        return hasOwn.call(obj, prop);
    }

    function getOwn(obj, prop) {
        return hasProp(obj, prop) && obj[prop];
    }

    /**
     * Cycles over properties in an object and calls a function for each
     * property value. If the function returns a truthy value, then the
     * iteration is stopped.
     */
    function eachProp(obj, func) {
        var prop;
        for (prop in obj) {
            if (hasProp(obj, prop)) {
                if (func(obj[prop], prop)) {
                    break;
                }
            }
        }
    }

    /**
     * Simple function to mix in properties from source into target,
     * but only if target does not already have a property of the same name.
     */
    function mixin(target, source, force, deepStringMixin) {
        if (source) {
            eachProp(source, function (value, prop) {
                if (force || !hasProp(target, prop)) {
                    if (deepStringMixin && typeof value === 'object' && value &&
                        !isArray(value) && !isFunction(value) &&
                        !(value instanceof RegExp)) {

                        if (!target[prop]) {
                            target[prop] = {};
                        }
                        mixin(target[prop], value, force, deepStringMixin);
                    } else {
                        target[prop] = value;
                    }
                }
            });
        }
        return target;
    }

    //Similar to Function.prototype.bind, but the 'this' object is specified
    //first, since it is easier to read/figure out what 'this' will be.
    function bind(obj, fn) {
        return function () {
            return fn.apply(obj, arguments);
        };
    }

    function scripts() {
        return document.getElementsByTagName('script');
    }

    function defaultOnError(err) {
        throw err;
    }

    //Allow getting a global that is expressed in
    //dot notation, like 'a.b.c'.
    function getGlobal(value) {
        if (!value) {
            return value;
        }
        var g = global;
        each(value.split('.'), function (part) {
            g = g[part];
        });
        return g;
    }

    /**
     * Constructs an error with a pointer to an URL with more information.
     * @param {String} id the error ID that maps to an ID on a web page.
     * @param {String} message human readable error.
     * @param {Error} [err] the original error, if there is one.
     *
     * @returns {Error}
     */
    function makeError(id, msg, err, requireModules) {
        var e = new Error(msg + '\nhttps://requirejs.org/docs/errors.html#' + id);
        e.requireType = id;
        e.requireModules = requireModules;
        if (err) {
            e.originalError = err;
        }
        return e;
    }

    if (typeof define !== 'undefined') {
        //If a define is already in play via another AMD loader,
        //do not overwrite.
        return;
    }

    if (typeof requirejs !== 'undefined') {
        if (isFunction(requirejs)) {
            //Do not overwrite an existing requirejs instance.
            return;
        }
        cfg = requirejs;
        requirejs = undefined;
    }

    //Allow for a require config object
    if (typeof require !== 'undefined' && !isFunction(require)) {
        //assume it is a config object.
        cfg = require;
        require = undefined;
    }

    function newContext(contextName) {
        var inCheckLoaded, Module, context, handlers,
            checkLoadedTimeoutId,
            config = {
                //Defaults. Do not set a default for map
                //config to speed up normalize(), which
                //will run faster if there is no default.
                waitSeconds: 7,
                baseUrl: './',
                paths: {},
                bundles: {},
                pkgs: {},
                shim: {},
                config: {}
            },
            registry = {},
            //registry of just enabled modules, to speed
            //cycle breaking code when lots of modules
            //are registered, but not activated.
            enabledRegistry = {},
            undefEvents = {},
            defQueue = [],
            defined = {},
            urlFetched = {},
            bundlesMap = {},
            requireCounter = 1,
            unnormalizedCounter = 1;

        /**
         * Trims the . and .. from an array of path segments.
         * It will keep a leading path segment if a .. will become
         * the first path segment, to help with module name lookups,
         * which act like paths, but can be remapped. But the end result,
         * all paths that use this function should look normalized.
         * NOTE: this method MODIFIES the input array.
         * @param {Array} ary the array of path segments.
         */
        function trimDots(ary) {
            var i, part;
            for (i = 0; i < ary.length; i++) {
                part = ary[i];
                if (part === '.') {
                    ary.splice(i, 1);
                    i -= 1;
                } else if (part === '..') {
                    // If at the start, or previous value is still ..,
                    // keep them so that when converted to a path it may
                    // still work when converted to a path, even though
                    // as an ID it is less than ideal. In larger point
                    // releases, may be better to just kick out an error.
                    if (i === 0 || (i === 1 && ary[2] === '..') || ary[i - 1] === '..') {
                        continue;
                    } else if (i > 0) {
                        ary.splice(i - 1, 2);
                        i -= 2;
                    }
                }
            }
        }

        /**
         * Given a relative module name, like ./something, normalize it to
         * a real name that can be mapped to a path.
         * @param {String} name the relative name
         * @param {String} baseName a real name that the name arg is relative
         * to.
         * @param {Boolean} applyMap apply the map config to the value. Should
         * only be done if this normalization is for a dependency ID.
         * @returns {String} normalized name
         */
        function normalize(name, baseName, applyMap) {
            var pkgMain, mapValue, nameParts, i, j, nameSegment, lastIndex,
                foundMap, foundI, foundStarMap, starI, normalizedBaseParts,
                baseParts = (baseName && baseName.split('/')),
                map = config.map,
                starMap = map && map['*'];

            //Adjust any relative paths.
            if (name) {
                name = name.split('/');
                lastIndex = name.length - 1;

                // If wanting node ID compatibility, strip .js from end
                // of IDs. Have to do this here, and not in nameToUrl
                // because node allows either .js or non .js to map
                // to same file.
                if (config.nodeIdCompat && jsSuffixRegExp.test(name[lastIndex])) {
                    name[lastIndex] = name[lastIndex].replace(jsSuffixRegExp, '');
                }

                // Starts with a '.' so need the baseName
                if (name[0].charAt(0) === '.' && baseParts) {
                    //Convert baseName to array, and lop off the last part,
                    //so that . matches that 'directory' and not name of the baseName's
                    //module. For instance, baseName of 'one/two/three', maps to
                    //'one/two/three.js', but we want the directory, 'one/two' for
                    //this normalization.
                    normalizedBaseParts = baseParts.slice(0, baseParts.length - 1);
                    name = normalizedBaseParts.concat(name);
                }

                trimDots(name);
                name = name.join('/');
            }

            //Apply map config if available.
            if (applyMap && map && (baseParts || starMap)) {
                nameParts = name.split('/');

                outerLoop: for (i = nameParts.length; i > 0; i -= 1) {
                    nameSegment = nameParts.slice(0, i).join('/');

                    if (baseParts) {
                        //Find the longest baseName segment match in the config.
                        //So, do joins on the biggest to smallest lengths of baseParts.
                        for (j = baseParts.length; j > 0; j -= 1) {
                            mapValue = getOwn(map, baseParts.slice(0, j).join('/'));

                            //baseName segment has config, find if it has one for
                            //this name.
                            if (mapValue) {
                                mapValue = getOwn(mapValue, nameSegment);
                                if (mapValue) {
                                    //Match, update name to the new value.
                                    foundMap = mapValue;
                                    foundI = i;
                                    break outerLoop;
                                }
                            }
                        }
                    }

                    //Check for a star map match, but just hold on to it,
                    //if there is a shorter segment match later in a matching
                    //config, then favor over this star map.
                    if (!foundStarMap && starMap && getOwn(starMap, nameSegment)) {
                        foundStarMap = getOwn(starMap, nameSegment);
                        starI = i;
                    }
                }

                if (!foundMap && foundStarMap) {
                    foundMap = foundStarMap;
                    foundI = starI;
                }

                if (foundMap) {
                    nameParts.splice(0, foundI, foundMap);
                    name = nameParts.join('/');
                }
            }

            // If the name points to a package's name, use
            // the package main instead.
            pkgMain = getOwn(config.pkgs, name);

            return pkgMain ? pkgMain : name;
        }

        function removeScript(name) {
            if (isBrowser) {
                each(scripts(), function (scriptNode) {
                    if (scriptNode.getAttribute('data-requiremodule') === name &&
                            scriptNode.getAttribute('data-requirecontext') === context.contextName) {
                        scriptNode.parentNode.removeChild(scriptNode);
                        return true;
                    }
                });
            }
        }

        function hasPathFallback(id) {
            var pathConfig = getOwn(config.paths, id);
            if (pathConfig && isArray(pathConfig) && pathConfig.length > 1) {
                //Pop off the first array value, since it failed, and
                //retry
                pathConfig.shift();
                context.require.undef(id);

                //Custom require that does not do map translation, since
                //ID is "absolute", already mapped/resolved.
                context.makeRequire(null, {
                    skipMap: true
                })([id]);

                return true;
            }
        }

        //Turns a plugin!resource to [plugin, resource]
        //with the plugin being undefined if the name
        //did not have a plugin prefix.
        function splitPrefix(name) {
            var prefix,
                index = name ? name.indexOf('!') : -1;
            if (index > -1) {
                prefix = name.substring(0, index);
                name = name.substring(index + 1, name.length);
            }
            return [prefix, name];
        }

        /**
         * Creates a module mapping that includes plugin prefix, module
         * name, and path. If parentModuleMap is provided it will
         * also normalize the name via require.normalize()
         *
         * @param {String} name the module name
         * @param {String} [parentModuleMap] parent module map
         * for the module name, used to resolve relative names.
         * @param {Boolean} isNormalized: is the ID already normalized.
         * This is true if this call is done for a define() module ID.
         * @param {Boolean} applyMap: apply the map config to the ID.
         * Should only be true if this map is for a dependency.
         *
         * @returns {Object}
         */
        function makeModuleMap(name, parentModuleMap, isNormalized, applyMap) {
            var url, pluginModule, suffix, nameParts,
                prefix = null,
                parentName = parentModuleMap ? parentModuleMap.name : null,
                originalName = name,
                isDefine = true,
                normalizedName = '';

            //If no name, then it means it is a require call, generate an
            //internal name.
            if (!name) {
                isDefine = false;
                name = '_@r' + (requireCounter += 1);
            }

            nameParts = splitPrefix(name);
            prefix = nameParts[0];
            name = nameParts[1];

            if (prefix) {
                prefix = normalize(prefix, parentName, applyMap);
                pluginModule = getOwn(defined, prefix);
            }

            //Account for relative paths if there is a base name.
            if (name) {
                if (prefix) {
                    if (isNormalized) {
                        normalizedName = name;
                    } else if (pluginModule && pluginModule.normalize) {
                        //Plugin is loaded, use its normalize method.
                        normalizedName = pluginModule.normalize(name, function (name) {
                            return normalize(name, parentName, applyMap);
                        });
                    } else {
                        // If nested plugin references, then do not try to
                        // normalize, as it will not normalize correctly. This
                        // places a restriction on resourceIds, and the longer
                        // term solution is not to normalize until plugins are
                        // loaded and all normalizations to allow for async
                        // loading of a loader plugin. But for now, fixes the
                        // common uses. Details in #1131
                        normalizedName = name.indexOf('!') === -1 ?
                                         normalize(name, parentName, applyMap) :
                                         name;
                    }
                } else {
                    //A regular module.
                    normalizedName = normalize(name, parentName, applyMap);

                    //Normalized name may be a plugin ID due to map config
                    //application in normalize. The map config values must
                    //already be normalized, so do not need to redo that part.
                    nameParts = splitPrefix(normalizedName);
                    prefix = nameParts[0];
                    normalizedName = nameParts[1];
                    isNormalized = true;

                    url = context.nameToUrl(normalizedName);
                }
            }

            //If the id is a plugin id that cannot be determined if it needs
            //normalization, stamp it with a unique ID so two matching relative
            //ids that may conflict can be separate.
            suffix = prefix && !pluginModule && !isNormalized ?
                     '_unnormalized' + (unnormalizedCounter += 1) :
                     '';

            return {
                prefix: prefix,
                name: normalizedName,
                parentMap: parentModuleMap,
                unnormalized: !!suffix,
                url: url,
                originalName: originalName,
                isDefine: isDefine,
                id: (prefix ?
                        prefix + '!' + normalizedName :
                        normalizedName) + suffix
            };
        }

        function getModule(depMap) {
            var id = depMap.id,
                mod = getOwn(registry, id);

            if (!mod) {
                mod = registry[id] = new context.Module(depMap);
            }

            return mod;
        }

        function on(depMap, name, fn) {
            var id = depMap.id,
                mod = getOwn(registry, id);

            if (hasProp(defined, id) &&
                    (!mod || mod.defineEmitComplete)) {
                if (name === 'defined') {
                    fn(defined[id]);
                }
            } else {
                mod = getModule(depMap);
                if (mod.error && name === 'error') {
                    fn(mod.error);
                } else {
                    mod.on(name, fn);
                }
            }
        }

        function onError(err, errback) {
            var ids = err.requireModules,
                notified = false;

            if (errback) {
                errback(err);
            } else {
                each(ids, function (id) {
                    var mod = getOwn(registry, id);
                    if (mod) {
                        //Set error on module, so it skips timeout checks.
                        mod.error = err;
                        if (mod.events.error) {
                            notified = true;
                            mod.emit('error', err);
                        }
                    }
                });

                if (!notified) {
                    req.onError(err);
                }
            }
        }

        /**
         * Internal method to transfer globalQueue items to this context's
         * defQueue.
         */
        function takeGlobalQueue() {
            //Push all the globalDefQueue items into the context's defQueue
            if (globalDefQueue.length) {
                each(globalDefQueue, function(queueItem) {
                    var id = queueItem[0];
                    if (typeof id === 'string') {
                        context.defQueueMap[id] = true;
                    }
                    defQueue.push(queueItem);
                });
                globalDefQueue = [];
            }
        }

        handlers = {
            'require': function (mod) {
                if (mod.require) {
                    return mod.require;
                } else {
                    return (mod.require = context.makeRequire(mod.map));
                }
            },
            'exports': function (mod) {
                mod.usingExports = true;
                if (mod.map.isDefine) {
                    if (mod.exports) {
                        return (defined[mod.map.id] = mod.exports);
                    } else {
                        return (mod.exports = defined[mod.map.id] = {});
                    }
                }
            },
            'module': function (mod) {
                if (mod.module) {
                    return mod.module;
                } else {
                    return (mod.module = {
                        id: mod.map.id,
                        uri: mod.map.url,
                        config: function () {
                            return getOwn(config.config, mod.map.id) || {};
                        },
                        exports: mod.exports || (mod.exports = {})
                    });
                }
            }
        };

        function cleanRegistry(id) {
            //Clean up machinery used for waiting modules.
            delete registry[id];
            delete enabledRegistry[id];
        }

        function breakCycle(mod, traced, processed) {
            var id = mod.map.id;

            if (mod.error) {
                mod.emit('error', mod.error);
            } else {
                traced[id] = true;
                each(mod.depMaps, function (depMap, i) {
                    var depId = depMap.id,
                        dep = getOwn(registry, depId);

                    //Only force things that have not completed
                    //being defined, so still in the registry,
                    //and only if it has not been matched up
                    //in the module already.
                    if (dep && !mod.depMatched[i] && !processed[depId]) {
                        if (getOwn(traced, depId)) {
                            mod.defineDep(i, defined[depId]);
                            mod.check(); //pass false?
                        } else {
                            breakCycle(dep, traced, processed);
                        }
                    }
                });
                processed[id] = true;
            }
        }

        function checkLoaded() {
            var err, usingPathFallback,
                waitInterval = config.waitSeconds * 1000,
                //It is possible to disable the wait interval by using waitSeconds of 0.
                expired = waitInterval && (context.startTime + waitInterval) < new Date().getTime(),
                noLoads = [],
                reqCalls = [],
                stillLoading = false,
                needCycleCheck = true;

            //Do not bother if this call was a result of a cycle break.
            if (inCheckLoaded) {
                return;
            }

            inCheckLoaded = true;

            //Figure out the state of all the modules.
            eachProp(enabledRegistry, function (mod) {
                var map = mod.map,
                    modId = map.id;

                //Skip things that are not enabled or in error state.
                if (!mod.enabled) {
                    return;
                }

                if (!map.isDefine) {
                    reqCalls.push(mod);
                }

                if (!mod.error) {
                    //If the module should be executed, and it has not
                    //been inited and time is up, remember it.
                    if (!mod.inited && expired) {
                        if (hasPathFallback(modId)) {
                            usingPathFallback = true;
                            stillLoading = true;
                        } else {
                            noLoads.push(modId);
                            removeScript(modId);
                        }
                    } else if (!mod.inited && mod.fetched && map.isDefine) {
                        stillLoading = true;
                        if (!map.prefix) {
                            //No reason to keep looking for unfinished
                            //loading. If the only stillLoading is a
                            //plugin resource though, keep going,
                            //because it may be that a plugin resource
                            //is waiting on a non-plugin cycle.
                            return (needCycleCheck = false);
                        }
                    }
                }
            });

            if (expired && noLoads.length) {
                //If wait time expired, throw error of unloaded modules.
                err = makeError('timeout', 'Load timeout for modules: ' + noLoads, null, noLoads);
                err.contextName = context.contextName;
                return onError(err);
            }

            //Not expired, check for a cycle.
            if (needCycleCheck) {
                each(reqCalls, function (mod) {
                    breakCycle(mod, {}, {});
                });
            }

            //If still waiting on loads, and the waiting load is something
            //other than a plugin resource, or there are still outstanding
            //scripts, then just try back later.
            if ((!expired || usingPathFallback) && stillLoading) {
                //Something is still waiting to load. Wait for it, but only
                //if a timeout is not already in effect.
                if ((isBrowser || isWebWorker) && !checkLoadedTimeoutId) {
                    checkLoadedTimeoutId = setTimeout(function () {
                        checkLoadedTimeoutId = 0;
                        checkLoaded();
                    }, 50);
                }
            }

            inCheckLoaded = false;
        }

        Module = function (map) {
            this.events = getOwn(undefEvents, map.id) || {};
            this.map = map;
            this.shim = getOwn(config.shim, map.id);
            this.depExports = [];
            this.depMaps = [];
            this.depMatched = [];
            this.pluginMaps = {};
            this.depCount = 0;

            /* this.exports this.factory
               this.depMaps = [],
               this.enabled, this.fetched
            */
        };

        Module.prototype = {
            init: function (depMaps, factory, errback, options) {
                options = options || {};

                //Do not do more inits if already done. Can happen if there
                //are multiple define calls for the same module. That is not
                //a normal, common case, but it is also not unexpected.
                if (this.inited) {
                    return;
                }

                this.factory = factory;

                if (errback) {
                    //Register for errors on this module.
                    this.on('error', errback);
                } else if (this.events.error) {
                    //If no errback already, but there are error listeners
                    //on this module, set up an errback to pass to the deps.
                    errback = bind(this, function (err) {
                        this.emit('error', err);
                    });
                }

                //Do a copy of the dependency array, so that
                //source inputs are not modified. For example
                //"shim" deps are passed in here directly, and
                //doing a direct modification of the depMaps array
                //would affect that config.
                this.depMaps = depMaps && depMaps.slice(0);

                this.errback = errback;

                //Indicate this module has be initialized
                this.inited = true;

                this.ignore = options.ignore;

                //Could have option to init this module in enabled mode,
                //or could have been previously marked as enabled. However,
                //the dependencies are not known until init is called. So
                //if enabled previously, now trigger dependencies as enabled.
                if (options.enabled || this.enabled) {
                    //Enable this module and dependencies.
                    //Will call this.check()
                    this.enable();
                } else {
                    this.check();
                }
            },

            defineDep: function (i, depExports) {
                //Because of cycles, defined callback for a given
                //export can be called more than once.
                if (!this.depMatched[i]) {
                    this.depMatched[i] = true;
                    this.depCount -= 1;
                    this.depExports[i] = depExports;
                }
            },

            fetch: function () {
                if (this.fetched) {
                    return;
                }
                this.fetched = true;

                context.startTime = (new Date()).getTime();

                var map = this.map;

                //If the manager is for a plugin managed resource,
                //ask the plugin to load it now.
                if (this.shim) {
                    context.makeRequire(this.map, {
                        enableBuildCallback: true
                    })(this.shim.deps || [], bind(this, function () {
                        return map.prefix ? this.callPlugin() : this.load();
                    }));
                } else {
                    //Regular dependency.
                    return map.prefix ? this.callPlugin() : this.load();
                }
            },

            load: function () {
                var url = this.map.url;

                //Regular dependency.
                if (!urlFetched[url]) {
                    urlFetched[url] = true;
                    context.load(this.map.id, url);
                }
            },

            /**
             * Checks if the module is ready to define itself, and if so,
             * define it.
             */
            check: function () {
                if (!this.enabled || this.enabling) {
                    return;
                }

                var err, cjsModule,
                    id = this.map.id,
                    depExports = this.depExports,
                    exports = this.exports,
                    factory = this.factory;

                if (!this.inited) {
                    // Only fetch if not already in the defQueue.
                    if (!hasProp(context.defQueueMap, id)) {
                        this.fetch();
                    }
                } else if (this.error) {
                    this.emit('error', this.error);
                } else if (!this.defining) {
                    //The factory could trigger another require call
                    //that would result in checking this module to
                    //define itself again. If already in the process
                    //of doing that, skip this work.
                    this.defining = true;

                    if (this.depCount < 1 && !this.defined) {
                        if (isFunction(factory)) {
                            //If there is an error listener, favor passing
                            //to that instead of throwing an error. However,
                            //only do it for define()'d  modules. require
                            //errbacks should not be called for failures in
                            //their callbacks (#699). However if a global
                            //onError is set, use that.
                            if ((this.events.error && this.map.isDefine) ||
                                req.onError !== defaultOnError) {
                                try {
                                    exports = context.execCb(id, factory, depExports, exports);
                                } catch (e) {
                                    err = e;
                                }
                            } else {
                                exports = context.execCb(id, factory, depExports, exports);
                            }

                            // Favor return value over exports. If node/cjs in play,
                            // then will not have a return value anyway. Favor
                            // module.exports assignment over exports object.
                            if (this.map.isDefine && exports === undefined) {
                                cjsModule = this.module;
                                if (cjsModule) {
                                    exports = cjsModule.exports;
                                } else if (this.usingExports) {
                                    //exports already set the defined value.
                                    exports = this.exports;
                                }
                            }

                            if (err) {
                                err.requireMap = this.map;
                                err.requireModules = this.map.isDefine ? [this.map.id] : null;
                                err.requireType = this.map.isDefine ? 'define' : 'require';
                                return onError((this.error = err));
                            }

                        } else {
                            //Just a literal value
                            exports = factory;
                        }

                        this.exports = exports;

                        if (this.map.isDefine && !this.ignore) {
                            defined[id] = exports;

                            if (req.onResourceLoad) {
                                var resLoadMaps = [];
                                each(this.depMaps, function (depMap) {
                                    resLoadMaps.push(depMap.normalizedMap || depMap);
                                });
                                req.onResourceLoad(context, this.map, resLoadMaps);
                            }
                        }

                        //Clean up
                        cleanRegistry(id);

                        this.defined = true;
                    }

                    //Finished the define stage. Allow calling check again
                    //to allow define notifications below in the case of a
                    //cycle.
                    this.defining = false;

                    if (this.defined && !this.defineEmitted) {
                        this.defineEmitted = true;
                        this.emit('defined', this.exports);
                        this.defineEmitComplete = true;
                    }

                }
            },

            callPlugin: function () {
                var map = this.map,
                    id = map.id,
                    //Map already normalized the prefix.
                    pluginMap = makeModuleMap(map.prefix);

                //Mark this as a dependency for this plugin, so it
                //can be traced for cycles.
                this.depMaps.push(pluginMap);

                on(pluginMap, 'defined', bind(this, function (plugin) {
                    var load, normalizedMap, normalizedMod,
                        bundleId = getOwn(bundlesMap, this.map.id),
                        name = this.map.name,
                        parentName = this.map.parentMap ? this.map.parentMap.name : null,
                        localRequire = context.makeRequire(map.parentMap, {
                            enableBuildCallback: true
                        });

                    //If current map is not normalized, wait for that
                    //normalized name to load instead of continuing.
                    if (this.map.unnormalized) {
                        //Normalize the ID if the plugin allows it.
                        if (plugin.normalize) {
                            name = plugin.normalize(name, function (name) {
                                return normalize(name, parentName, true);
                            }) || '';
                        }

                        //prefix and name should already be normalized, no need
                        //for applying map config again either.
                        normalizedMap = makeModuleMap(map.prefix + '!' + name,
                                                      this.map.parentMap,
                                                      true);
                        on(normalizedMap,
                            'defined', bind(this, function (value) {
                                this.map.normalizedMap = normalizedMap;
                                this.init([], function () { return value; }, null, {
                                    enabled: true,
                                    ignore: true
                                });
                            }));

                        normalizedMod = getOwn(registry, normalizedMap.id);
                        if (normalizedMod) {
                            //Mark this as a dependency for this plugin, so it
                            //can be traced for cycles.
                            this.depMaps.push(normalizedMap);

                            if (this.events.error) {
                                normalizedMod.on('error', bind(this, function (err) {
                                    this.emit('error', err);
                                }));
                            }
                            normalizedMod.enable();
                        }

                        return;
                    }

                    //If a paths config, then just load that file instead to
                    //resolve the plugin, as it is built into that paths layer.
                    if (bundleId) {
                        this.map.url = context.nameToUrl(bundleId);
                        this.load();
                        return;
                    }

                    load = bind(this, function (value) {
                        this.init([], function () { return value; }, null, {
                            enabled: true
                        });
                    });

                    load.error = bind(this, function (err) {
                        this.inited = true;
                        this.error = err;
                        err.requireModules = [id];

                        //Remove temp unnormalized modules for this module,
                        //since they will never be resolved otherwise now.
                        eachProp(registry, function (mod) {
                            if (mod.map.id.indexOf(id + '_unnormalized') === 0) {
                                cleanRegistry(mod.map.id);
                            }
                        });

                        onError(err);
                    });

                    //Allow plugins to load other code without having to know the
                    //context or how to 'complete' the load.
                    load.fromText = bind(this, function (text, textAlt) {
                        /*jslint evil: true */
                        var moduleName = map.name,
                            moduleMap = makeModuleMap(moduleName),
                            hasInteractive = useInteractive;

                        //As of 2.1.0, support just passing the text, to reinforce
                        //fromText only being called once per resource. Still
                        //support old style of passing moduleName but discard
                        //that moduleName in favor of the internal ref.
                        if (textAlt) {
                            text = textAlt;
                        }

                        //Turn off interactive script matching for IE for any define
                        //calls in the text, then turn it back on at the end.
                        if (hasInteractive) {
                            useInteractive = false;
                        }

                        //Prime the system by creating a module instance for
                        //it.
                        getModule(moduleMap);

                        //Transfer any config to this other module.
                        if (hasProp(config.config, id)) {
                            config.config[moduleName] = config.config[id];
                        }

                        try {
                            req.exec(text);
                        } catch (e) {
                            return onError(makeError('fromtexteval',
                                             'fromText eval for ' + id +
                                            ' failed: ' + e,
                                             e,
                                             [id]));
                        }

                        if (hasInteractive) {
                            useInteractive = true;
                        }

                        //Mark this as a dependency for the plugin
                        //resource
                        this.depMaps.push(moduleMap);

                        //Support anonymous modules.
                        context.completeLoad(moduleName);

                        //Bind the value of that module to the value for this
                        //resource ID.
                        localRequire([moduleName], load);
                    });

                    //Use parentName here since the plugin's name is not reliable,
                    //could be some weird string with no path that actually wants to
                    //reference the parentName's path.
                    plugin.load(map.name, localRequire, load, config);
                }));

                context.enable(pluginMap, this);
                this.pluginMaps[pluginMap.id] = pluginMap;
            },

            enable: function () {
                enabledRegistry[this.map.id] = this;
                this.enabled = true;

                //Set flag mentioning that the module is enabling,
                //so that immediate calls to the defined callbacks
                //for dependencies do not trigger inadvertent load
                //with the depCount still being zero.
                this.enabling = true;

                //Enable each dependency
                each(this.depMaps, bind(this, function (depMap, i) {
                    var id, mod, handler;

                    if (typeof depMap === 'string') {
                        //Dependency needs to be converted to a depMap
                        //and wired up to this module.
                        depMap = makeModuleMap(depMap,
                                               (this.map.isDefine ? this.map : this.map.parentMap),
                                               false,
                                               !this.skipMap);
                        this.depMaps[i] = depMap;

                        handler = getOwn(handlers, depMap.id);

                        if (handler) {
                            this.depExports[i] = handler(this);
                            return;
                        }

                        this.depCount += 1;

                        on(depMap, 'defined', bind(this, function (depExports) {
                            if (this.undefed) {
                                return;
                            }
                            this.defineDep(i, depExports);
                            this.check();
                        }));

                        if (this.errback) {
                            on(depMap, 'error', bind(this, this.errback));
                        } else if (this.events.error) {
                            // No direct errback on this module, but something
                            // else is listening for errors, so be sure to
                            // propagate the error correctly.
                            on(depMap, 'error', bind(this, function(err) {
                                this.emit('error', err);
                            }));
                        }
                    }

                    id = depMap.id;
                    mod = registry[id];

                    //Skip special modules like 'require', 'exports', 'module'
                    //Also, don't call enable if it is already enabled,
                    //important in circular dependency cases.
                    if (!hasProp(handlers, id) && mod && !mod.enabled) {
                        context.enable(depMap, this);
                    }
                }));

                //Enable each plugin that is used in
                //a dependency
                eachProp(this.pluginMaps, bind(this, function (pluginMap) {
                    var mod = getOwn(registry, pluginMap.id);
                    if (mod && !mod.enabled) {
                        context.enable(pluginMap, this);
                    }
                }));

                this.enabling = false;

                this.check();
            },

            on: function (name, cb) {
                var cbs = this.events[name];
                if (!cbs) {
                    cbs = this.events[name] = [];
                }
                cbs.push(cb);
            },

            emit: function (name, evt) {
                each(this.events[name], function (cb) {
                    cb(evt);
                });
                if (name === 'error') {
                    //Now that the error handler was triggered, remove
                    //the listeners, since this broken Module instance
                    //can stay around for a while in the registry.
                    delete this.events[name];
                }
            }
        };

        function callGetModule(args) {
            //Skip modules already defined.
            if (!hasProp(defined, args[0])) {
                getModule(makeModuleMap(args[0], null, true)).init(args[1], args[2]);
            }
        }

        function removeListener(node, func, name, ieName) {
            //Favor detachEvent because of IE9
            //issue, see attachEvent/addEventListener comment elsewhere
            //in this file.
            if (node.detachEvent && !isOpera) {
                //Probably IE. If not it will throw an error, which will be
                //useful to know.
                if (ieName) {
                    node.detachEvent(ieName, func);
                }
            } else {
                node.removeEventListener(name, func, false);
            }
        }

        /**
         * Given an event from a script node, get the requirejs info from it,
         * and then removes the event listeners on the node.
         * @param {Event} evt
         * @returns {Object}
         */
        function getScriptData(evt) {
            //Using currentTarget instead of target for Firefox 2.0's sake. Not
            //all old browsers will be supported, but this one was easy enough
            //to support and still makes sense.
            var node = evt.currentTarget || evt.srcElement;

            //Remove the listeners once here.
            removeListener(node, context.onScriptLoad, 'load', 'onreadystatechange');
            removeListener(node, context.onScriptError, 'error');

            return {
                node: node,
                id: node && node.getAttribute('data-requiremodule')
            };
        }

        function intakeDefines() {
            var args;

            //Any defined modules in the global queue, intake them now.
            takeGlobalQueue();

            //Make sure any remaining defQueue items get properly processed.
            while (defQueue.length) {
                args = defQueue.shift();
                if (args[0] === null) {
                    return onError(makeError('mismatch', 'Mismatched anonymous define() module: ' +
                        args[args.length - 1]));
                } else {
                    //args are id, deps, factory. Should be normalized by the
                    //define() function.
                    callGetModule(args);
                }
            }
            context.defQueueMap = {};
        }

        context = {
            config: config,
            contextName: contextName,
            registry: registry,
            defined: defined,
            urlFetched: urlFetched,
            defQueue: defQueue,
            defQueueMap: {},
            Module: Module,
            makeModuleMap: makeModuleMap,
            nextTick: req.nextTick,
            onError: onError,

            /**
             * Set a configuration for the context.
             * @param {Object} cfg config object to integrate.
             */
            configure: function (cfg) {
                //Make sure the baseUrl ends in a slash.
                if (cfg.baseUrl) {
                    if (cfg.baseUrl.charAt(cfg.baseUrl.length - 1) !== '/') {
                        cfg.baseUrl += '/';
                    }
                }

                // Convert old style urlArgs string to a function.
                if (typeof cfg.urlArgs === 'string') {
                    var urlArgs = cfg.urlArgs;
                    cfg.urlArgs = function(id, url) {
                        return (url.indexOf('?') === -1 ? '?' : '&') + urlArgs;
                    };
                }

                //Save off the paths since they require special processing,
                //they are additive.
                var shim = config.shim,
                    objs = {
                        paths: true,
                        bundles: true,
                        config: true,
                        map: true
                    };

                eachProp(cfg, function (value, prop) {
                    if (objs[prop]) {
                        if (!config[prop]) {
                            config[prop] = {};
                        }
                        mixin(config[prop], value, true, true);
                    } else {
                        config[prop] = value;
                    }
                });

                //Reverse map the bundles
                if (cfg.bundles) {
                    eachProp(cfg.bundles, function (value, prop) {
                        each(value, function (v) {
                            if (v !== prop) {
                                bundlesMap[v] = prop;
                            }
                        });
                    });
                }

                //Merge shim
                if (cfg.shim) {
                    eachProp(cfg.shim, function (value, id) {
                        //Normalize the structure
                        if (isArray(value)) {
                            value = {
                                deps: value
                            };
                        }
                        if ((value.exports || value.init) && !value.exportsFn) {
                            value.exportsFn = context.makeShimExports(value);
                        }
                        shim[id] = value;
                    });
                    config.shim = shim;
                }

                //Adjust packages if necessary.
                if (cfg.packages) {
                    each(cfg.packages, function (pkgObj) {
                        var location, name;

                        pkgObj = typeof pkgObj === 'string' ? {name: pkgObj} : pkgObj;

                        name = pkgObj.name;
                        location = pkgObj.location;
                        if (location) {
                            config.paths[name] = pkgObj.location;
                        }

                        //Save pointer to main module ID for pkg name.
                        //Remove leading dot in main, so main paths are normalized,
                        //and remove any trailing .js, since different package
                        //envs have different conventions: some use a module name,
                        //some use a file name.
                        config.pkgs[name] = pkgObj.name + '/' + (pkgObj.main || 'main')
                                     .replace(currDirRegExp, '')
                                     .replace(jsSuffixRegExp, '');
                    });
                }

                //If there are any "waiting to execute" modules in the registry,
                //update the maps for them, since their info, like URLs to load,
                //may have changed.
                eachProp(registry, function (mod, id) {
                    //If module already has init called, since it is too
                    //late to modify them, and ignore unnormalized ones
                    //since they are transient.
                    if (!mod.inited && !mod.map.unnormalized) {
                        mod.map = makeModuleMap(id, null, true);
                    }
                });

                //If a deps array or a config callback is specified, then call
                //require with those args. This is useful when require is defined as a
                //config object before require.js is loaded.
                if (cfg.deps || cfg.callback) {
                    context.require(cfg.deps || [], cfg.callback);
                }
            },

            makeShimExports: function (value) {
                function fn() {
                    var ret;
                    if (value.init) {
                        ret = value.init.apply(global, arguments);
                    }
                    return ret || (value.exports && getGlobal(value.exports));
                }
                return fn;
            },

            makeRequire: function (relMap, options) {
                options = options || {};

                function localRequire(deps, callback, errback) {
                    var id, map, requireMod;

                    if (options.enableBuildCallback && callback && isFunction(callback)) {
                        callback.__requireJsBuild = true;
                    }

                    if (typeof deps === 'string') {
                        if (isFunction(callback)) {
                            //Invalid call
                            return onError(makeError('requireargs', 'Invalid require call'), errback);
                        }

                        //If require|exports|module are requested, get the
                        //value for them from the special handlers. Caveat:
                        //this only works while module is being defined.
                        if (relMap && hasProp(handlers, deps)) {
                            return handlers[deps](registry[relMap.id]);
                        }

                        //Synchronous access to one module. If require.get is
                        //available (as in the Node adapter), prefer that.
                        if (req.get) {
                            return req.get(context, deps, relMap, localRequire);
                        }

                        //Normalize module name, if it contains . or ..
                        map = makeModuleMap(deps, relMap, false, true);
                        id = map.id;

                        if (!hasProp(defined, id)) {
                            return onError(makeError('notloaded', 'Module name "' +
                                        id +
                                        '" has not been loaded yet for context: ' +
                                        contextName +
                                        (relMap ? '' : '. Use require([])')));
                        }
                        return defined[id];
                    }

                    //Grab defines waiting in the global queue.
                    intakeDefines();

                    //Mark all the dependencies as needing to be loaded.
                    context.nextTick(function () {
                        //Some defines could have been added since the
                        //require call, collect them.
                        intakeDefines();

                        requireMod = getModule(makeModuleMap(null, relMap));

                        //Store if map config should be applied to this require
                        //call for dependencies.
                        requireMod.skipMap = options.skipMap;

                        requireMod.init(deps, callback, errback, {
                            enabled: true
                        });

                        checkLoaded();
                    });

                    return localRequire;
                }

                mixin(localRequire, {
                    isBrowser: isBrowser,

                    /**
                     * Converts a module name + .extension into an URL path.
                     * *Requires* the use of a module name. It does not support using
                     * plain URLs like nameToUrl.
                     */
                    toUrl: function (moduleNamePlusExt) {
                        var ext,
                            index = moduleNamePlusExt.lastIndexOf('.'),
                            segment = moduleNamePlusExt.split('/')[0],
                            isRelative = segment === '.' || segment === '..';

                        //Have a file extension alias, and it is not the
                        //dots from a relative path.
                        if (index !== -1 && (!isRelative || index > 1)) {
                            ext = moduleNamePlusExt.substring(index, moduleNamePlusExt.length);
                            moduleNamePlusExt = moduleNamePlusExt.substring(0, index);
                        }

                        return context.nameToUrl(normalize(moduleNamePlusExt,
                                                relMap && relMap.id, true), ext,  true);
                    },

                    defined: function (id) {
                        return hasProp(defined, makeModuleMap(id, relMap, false, true).id);
                    },

                    specified: function (id) {
                        id = makeModuleMap(id, relMap, false, true).id;
                        return hasProp(defined, id) || hasProp(registry, id);
                    }
                });

                //Only allow undef on top level require calls
                if (!relMap) {
                    localRequire.undef = function (id) {
                        //Bind any waiting define() calls to this context,
                        //fix for #408
                        takeGlobalQueue();

                        var map = makeModuleMap(id, relMap, true),
                            mod = getOwn(registry, id);

                        mod.undefed = true;
                        removeScript(id);

                        delete defined[id];
                        delete urlFetched[map.url];
                        delete undefEvents[id];

                        //Clean queued defines too. Go backwards
                        //in array so that the splices do not
                        //mess up the iteration.
                        eachReverse(defQueue, function(args, i) {
                            if (args[0] === id) {
                                defQueue.splice(i, 1);
                            }
                        });
                        delete context.defQueueMap[id];

                        if (mod) {
                            //Hold on to listeners in case the
                            //module will be attempted to be reloaded
                            //using a different config.
                            if (mod.events.defined) {
                                undefEvents[id] = mod.events;
                            }

                            cleanRegistry(id);
                        }
                    };
                }

                return localRequire;
            },

            /**
             * Called to enable a module if it is still in the registry
             * awaiting enablement. A second arg, parent, the parent module,
             * is passed in for context, when this method is overridden by
             * the optimizer. Not shown here to keep code compact.
             */
            enable: function (depMap) {
                var mod = getOwn(registry, depMap.id);
                if (mod) {
                    getModule(depMap).enable();
                }
            },

            /**
             * Internal method used by environment adapters to complete a load event.
             * A load event could be a script load or just a load pass from a synchronous
             * load call.
             * @param {String} moduleName the name of the module to potentially complete.
             */
            completeLoad: function (moduleName) {
                var found, args, mod,
                    shim = getOwn(config.shim, moduleName) || {},
                    shExports = shim.exports;

                takeGlobalQueue();

                while (defQueue.length) {
                    args = defQueue.shift();
                    if (args[0] === null) {
                        args[0] = moduleName;
                        //If already found an anonymous module and bound it
                        //to this name, then this is some other anon module
                        //waiting for its completeLoad to fire.
                        if (found) {
                            break;
                        }
                        found = true;
                    } else if (args[0] === moduleName) {
                        //Found matching define call for this script!
                        found = true;
                    }

                    callGetModule(args);
                }
                context.defQueueMap = {};

                //Do this after the cycle of callGetModule in case the result
                //of those calls/init calls changes the registry.
                mod = getOwn(registry, moduleName);

                if (!found && !hasProp(defined, moduleName) && mod && !mod.inited) {
                    if (config.enforceDefine && (!shExports || !getGlobal(shExports))) {
                        if (hasPathFallback(moduleName)) {
                            return;
                        } else {
                            return onError(makeError('nodefine',
                                             'No define call for ' + moduleName,
                                             null,
                                             [moduleName]));
                        }
                    } else {
                        //A script that does not call define(), so just simulate
                        //the call for it.
                        callGetModule([moduleName, (shim.deps || []), shim.exportsFn]);
                    }
                }

                checkLoaded();
            },

            /**
             * Converts a module name to a file path. Supports cases where
             * moduleName may actually be just an URL.
             * Note that it **does not** call normalize on the moduleName,
             * it is assumed to have already been normalized. This is an
             * internal API, not a public one. Use toUrl for the public API.
             */
            nameToUrl: function (moduleName, ext, skipExt) {
                var paths, syms, i, parentModule, url,
                    parentPath, bundleId,
                    pkgMain = getOwn(config.pkgs, moduleName);

                if (pkgMain) {
                    moduleName = pkgMain;
                }

                bundleId = getOwn(bundlesMap, moduleName);

                if (bundleId) {
                    return context.nameToUrl(bundleId, ext, skipExt);
                }

                //If a colon is in the URL, it indicates a protocol is used and it is just
                //an URL to a file, or if it starts with a slash, contains a query arg (i.e. ?)
                //or ends with .js, then assume the user meant to use an url and not a module id.
                //The slash is important for protocol-less URLs as well as full paths.
                if (req.jsExtRegExp.test(moduleName)) {
                    //Just a plain path, not module name lookup, so just return it.
                    //Add extension if it is included. This is a bit wonky, only non-.js things pass
                    //an extension, this method probably needs to be reworked.
                    url = moduleName + (ext || '');
                } else {
                    //A module that needs to be converted to a path.
                    paths = config.paths;

                    syms = moduleName.split('/');
                    //For each module name segment, see if there is a path
                    //registered for it. Start with most specific name
                    //and work up from it.
                    for (i = syms.length; i > 0; i -= 1) {
                        parentModule = syms.slice(0, i).join('/');

                        parentPath = getOwn(paths, parentModule);
                        if (parentPath) {
                            //If an array, it means there are a few choices,
                            //Choose the one that is desired
                            if (isArray(parentPath)) {
                                parentPath = parentPath[0];
                            }
                            syms.splice(0, i, parentPath);
                            break;
                        }
                    }

                    //Join the path parts together, then figure out if baseUrl is needed.
                    url = syms.join('/');
                    url += (ext || (/^data\:|^blob\:|\?/.test(url) || skipExt ? '' : '.js'));
                    url = (url.charAt(0) === '/' || url.match(/^[\w\+\.\-]+:/) ? '' : config.baseUrl) + url;
                }

                return config.urlArgs && !/^blob\:/.test(url) ?
                       url + config.urlArgs(moduleName, url) : url;
            },

            //Delegates to req.load. Broken out as a separate function to
            //allow overriding in the optimizer.
            load: function (id, url) {
                req.load(context, id, url);
            },

            /**
             * Executes a module callback function. Broken out as a separate function
             * solely to allow the build system to sequence the files in the built
             * layer in the right sequence.
             *
             * @private
             */
            execCb: function (name, callback, args, exports) {
                return callback.apply(exports, args);
            },

            /**
             * callback for script loads, used to check status of loading.
             *
             * @param {Event} evt the event from the browser for the script
             * that was loaded.
             */
            onScriptLoad: function (evt) {
                //Using currentTarget instead of target for Firefox 2.0's sake. Not
                //all old browsers will be supported, but this one was easy enough
                //to support and still makes sense.
                if (evt.type === 'load' ||
                        (readyRegExp.test((evt.currentTarget || evt.srcElement).readyState))) {
                    //Reset interactive script so a script node is not held onto for
                    //to long.
                    interactiveScript = null;

                    //Pull out the name of the module and the context.
                    var data = getScriptData(evt);
                    context.completeLoad(data.id);
                }
            },

            /**
             * Callback for script errors.
             */
            onScriptError: function (evt) {
                var data = getScriptData(evt);
                if (!hasPathFallback(data.id)) {
                    var parents = [];
                    eachProp(registry, function(value, key) {
                        if (key.indexOf('_@r') !== 0) {
                            each(value.depMaps, function(depMap) {
                                if (depMap.id === data.id) {
                                    parents.push(key);
                                    return true;
                                }
                            });
                        }
                    });
                    return onError(makeError('scripterror', 'Script error for "' + data.id +
                                             (parents.length ?
                                             '", needed by: ' + parents.join(', ') :
                                             '"'), evt, [data.id]));
                }
            }
        };

        context.require = context.makeRequire();
        return context;
    }

    /**
     * Main entry point.
     *
     * If the only argument to require is a string, then the module that
     * is represented by that string is fetched for the appropriate context.
     *
     * If the first argument is an array, then it will be treated as an array
     * of dependency string names to fetch. An optional function callback can
     * be specified to execute when all of those dependencies are available.
     *
     * Make a local req variable to help Caja compliance (it assumes things
     * on a require that are not standardized), and to give a short
     * name for minification/local scope use.
     */
    req = requirejs = function (deps, callback, errback, optional) {

        //Find the right context, use default
        var context, config,
            contextName = defContextName;

        // Determine if have config object in the call.
        if (!isArray(deps) && typeof deps !== 'string') {
            // deps is a config object
            config = deps;
            if (isArray(callback)) {
                // Adjust args if there are dependencies
                deps = callback;
                callback = errback;
                errback = optional;
            } else {
                deps = [];
            }
        }

        if (config && config.context) {
            contextName = config.context;
        }

        context = getOwn(contexts, contextName);
        if (!context) {
            context = contexts[contextName] = req.s.newContext(contextName);
        }

        if (config) {
            context.configure(config);
        }

        return context.require(deps, callback, errback);
    };

    /**
     * Support require.config() to make it easier to cooperate with other
     * AMD loaders on globally agreed names.
     */
    req.config = function (config) {
        return req(config);
    };

    /**
     * Execute something after the current tick
     * of the event loop. Override for other envs
     * that have a better solution than setTimeout.
     * @param  {Function} fn function to execute later.
     */
    req.nextTick = typeof setTimeout !== 'undefined' ? function (fn) {
        setTimeout(fn, 4);
    } : function (fn) { fn(); };

    /**
     * Export require as a global, but only if it does not already exist.
     */
    if (!require) {
        require = req;
    }

    req.version = version;

    //Used to filter out dependencies that are already paths.
    req.jsExtRegExp = /^\/|:|\?|\.js$/;
    req.isBrowser = isBrowser;
    s = req.s = {
        contexts: contexts,
        newContext: newContext
    };

    //Create default context.
    req({});

    //Exports some context-sensitive methods on global require.
    each([
        'toUrl',
        'undef',
        'defined',
        'specified'
    ], function (prop) {
        //Reference from contexts instead of early binding to default context,
        //so that during builds, the latest instance of the default context
        //with its config gets used.
        req[prop] = function () {
            var ctx = contexts[defContextName];
            return ctx.require[prop].apply(ctx, arguments);
        };
    });

    if (isBrowser) {
        head = s.head = document.getElementsByTagName('head')[0];
        //If BASE tag is in play, using appendChild is a problem for IE6.
        //When that browser dies, this can be removed. Details in this jQuery bug:
        //http://dev.jquery.com/ticket/2709
        baseElement = document.getElementsByTagName('base')[0];
        if (baseElement) {
            head = s.head = baseElement.parentNode;
        }
    }

    /**
     * Any errors that require explicitly generates will be passed to this
     * function. Intercept/override it if you want custom error handling.
     * @param {Error} err the error object.
     */
    req.onError = defaultOnError;

    /**
     * Creates the node for the load command. Only used in browser envs.
     */
    req.createNode = function (config, moduleName, url) {
        var node = config.xhtml ?
                document.createElementNS('http://www.w3.org/1999/xhtml', 'html:script') :
                document.createElement('script');
        node.type = config.scriptType || 'text/javascript';
        node.charset = 'utf-8';
        node.async = true;
        return node;
    };

    /**
     * Does the request to load a module for the browser case.
     * Make this a separate function to allow other environments
     * to override it.
     *
     * @param {Object} context the require context to find state.
     * @param {String} moduleName the name of the module.
     * @param {Object} url the URL to the module.
     */
    req.load = function (context, moduleName, url) {
        var config = (context && context.config) || {},
            node;
        if (isBrowser) {
            //In the browser so use a script tag
            node = req.createNode(config, moduleName, url);

            node.setAttribute('data-requirecontext', context.contextName);
            node.setAttribute('data-requiremodule', moduleName);

            //Set up load listener. Test attachEvent first because IE9 has
            //a subtle issue in its addEventListener and script onload firings
            //that do not match the behavior of all other browsers with
            //addEventListener support, which fire the onload event for a
            //script right after the script execution. See:
            //https://connect.microsoft.com/IE/feedback/details/648057/script-onload-event-is-not-fired-immediately-after-script-execution
            //UNFORTUNATELY Opera implements attachEvent but does not follow the script
            //script execution mode.
            if (node.attachEvent &&
                    //Check if node.attachEvent is artificially added by custom script or
                    //natively supported by browser
                    //read https://github.com/requirejs/requirejs/issues/187
                    //if we can NOT find [native code] then it must NOT natively supported.
                    //in IE8, node.attachEvent does not have toString()
                    //Note the test for "[native code" with no closing brace, see:
                    //https://github.com/requirejs/requirejs/issues/273
                    !(node.attachEvent.toString && node.attachEvent.toString().indexOf('[native code') < 0) &&
                    !isOpera) {
                //Probably IE. IE (at least 6-8) do not fire
                //script onload right after executing the script, so
                //we cannot tie the anonymous define call to a name.
                //However, IE reports the script as being in 'interactive'
                //readyState at the time of the define call.
                useInteractive = true;

                node.attachEvent('onreadystatechange', context.onScriptLoad);
                //It would be great to add an error handler here to catch
                //404s in IE9+. However, onreadystatechange will fire before
                //the error handler, so that does not help. If addEventListener
                //is used, then IE will fire error before load, but we cannot
                //use that pathway given the connect.microsoft.com issue
                //mentioned above about not doing the 'script execute,
                //then fire the script load event listener before execute
                //next script' that other browsers do.
                //Best hope: IE10 fixes the issues,
                //and then destroys all installs of IE 6-9.
                //node.attachEvent('onerror', context.onScriptError);
            } else {
                node.addEventListener('load', context.onScriptLoad, false);
                node.addEventListener('error', context.onScriptError, false);
            }
            node.src = url;

            //Calling onNodeCreated after all properties on the node have been
            //set, but before it is placed in the DOM.
            if (config.onNodeCreated) {
                config.onNodeCreated(node, config, moduleName, url);
            }

            //For some cache cases in IE 6-8, the script executes before the end
            //of the appendChild execution, so to tie an anonymous define
            //call to the module name (which is stored on the node), hold on
            //to a reference to this node, but clear after the DOM insertion.
            currentlyAddingScript = node;
            if (baseElement) {
                head.insertBefore(node, baseElement);
            } else {
                head.appendChild(node);
            }
            currentlyAddingScript = null;

            return node;
        } else if (isWebWorker) {
            try {
                //In a web worker, use importScripts. This is not a very
                //efficient use of importScripts, importScripts will block until
                //its script is downloaded and evaluated. However, if web workers
                //are in play, the expectation is that a build has been done so
                //that only one script needs to be loaded anyway. This may need
                //to be reevaluated if other use cases become common.

                // Post a task to the event loop to work around a bug in WebKit
                // where the worker gets garbage-collected after calling
                // importScripts(): https://webkit.org/b/153317
                setTimeout(function() {}, 0);
                importScripts(url);

                //Account for anonymous modules
                context.completeLoad(moduleName);
            } catch (e) {
                context.onError(makeError('importscripts',
                                'importScripts failed for ' +
                                    moduleName + ' at ' + url,
                                e,
                                [moduleName]));
            }
        }
    };

    function getInteractiveScript() {
        if (interactiveScript && interactiveScript.readyState === 'interactive') {
            return interactiveScript;
        }

        eachReverse(scripts(), function (script) {
            if (script.readyState === 'interactive') {
                return (interactiveScript = script);
            }
        });
        return interactiveScript;
    }

    //Look for a data-main script attribute, which could also adjust the baseUrl.
    if (isBrowser && !cfg.skipDataMain) {
        //Figure out baseUrl. Get it from the script tag with require.js in it.
        eachReverse(scripts(), function (script) {
            //Set the 'head' where we can append children by
            //using the script's parent.
            if (!head) {
                head = script.parentNode;
            }

            //Look for a data-main attribute to set main script for the page
            //to load. If it is there, the path to data main becomes the
            //baseUrl, if it is not already set.
            dataMain = script.getAttribute('data-main');
            if (dataMain) {
                //Preserve dataMain in case it is a path (i.e. contains '?')
                mainScript = dataMain;

                //Set final baseUrl if there is not already an explicit one,
                //but only do so if the data-main value is not a loader plugin
                //module ID.
                if (!cfg.baseUrl && mainScript.indexOf('!') === -1) {
                    //Pull off the directory of data-main for use as the
                    //baseUrl.
                    src = mainScript.split('/');
                    mainScript = src.pop();
                    subPath = src.length ? src.join('/')  + '/' : './';

                    cfg.baseUrl = subPath;
                }

                //Strip off any trailing .js since mainScript is now
                //like a module name.
                mainScript = mainScript.replace(jsSuffixRegExp, '');

                //If mainScript is still a path, fall back to dataMain
                if (req.jsExtRegExp.test(mainScript)) {
                    mainScript = dataMain;
                }

                //Put the data-main script in the files to load.
                cfg.deps = cfg.deps ? cfg.deps.concat(mainScript) : [mainScript];

                return true;
            }
        });
    }

    /**
     * The function that handles definitions of modules. Differs from
     * require() in that a string for the module should be the first argument,
     * and the function to execute after dependencies are loaded should
     * return a value to define the module corresponding to the first argument's
     * name.
     */
    define = function (name, deps, callback) {
        var node, context;

        //Allow for anonymous modules
        if (typeof name !== 'string') {
            //Adjust args appropriately
            callback = deps;
            deps = name;
            name = null;
        }

        //This module may not have dependencies
        if (!isArray(deps)) {
            callback = deps;
            deps = null;
        }

        //If no name, and callback is a function, then figure out if it a
        //CommonJS thing with dependencies.
        if (!deps && isFunction(callback)) {
            deps = [];
            //Remove comments from the callback string,
            //look for require calls, and pull them into the dependencies,
            //but only if there are function args.
            if (callback.length) {
                callback
                    .toString()
                    .replace(commentRegExp, commentReplace)
                    .replace(cjsRequireRegExp, function (match, dep) {
                        deps.push(dep);
                    });

                //May be a CommonJS thing even without require calls, but still
                //could use exports, and module. Avoid doing exports and module
                //work though if it just needs require.
                //REQUIRES the function to expect the CommonJS variables in the
                //order listed below.
                deps = (callback.length === 1 ? ['require'] : ['require', 'exports', 'module']).concat(deps);
            }
        }

        //If in IE 6-8 and hit an anonymous define() call, do the interactive
        //work.
        if (useInteractive) {
            node = currentlyAddingScript || getInteractiveScript();
            if (node) {
                if (!name) {
                    name = node.getAttribute('data-requiremodule');
                }
                context = contexts[node.getAttribute('data-requirecontext')];
            }
        }

        //Always save off evaluating the def call until the script onload handler.
        //This allows multiple modules to be in a file without prematurely
        //tracing dependencies, and allows for anonymous module support,
        //where the module name is not known until the script onload event
        //occurs. If no context, use the global queue, and get it processed
        //in the onscript load callback.
        if (context) {
            context.defQueue.push([name, deps, callback]);
            context.defQueueMap[name] = true;
        } else {
            globalDefQueue.push([name, deps, callback]);
        }
    };

    define.amd = {
        jQuery: true
    };

    /**
     * Executes the text. Normally just uses eval, but can be modified
     * to use a better, environment-specific call. Only used for transpiling
     * loader plugins, not for plain JS modules.
     * @param {String} text the text to execute/evaluate.
     */
    req.exec = function (text) {
        /*jslint evil: true */
        return eval(text);
    };

    //Set up with config info.
    req(cfg);                                
}(this, (typeof setTimeout === 'undefined' ? undefined : setTimeout)));

In [ ]:
### File manipulation
def define_fold_or_file(*argv, file_name):
    if argv:
        folder = os.path.join(os.path.normpath(os.getcwd()))
        
        if argv[0].find(folder) != -1:
            file = os.path.join(argv[0], file_name)
            return file
        
        for i in range(len(argv)):
            folder = os.path.join(folder, argv[i])
        
        if file_name == 'None':
            return folder
        else:
            file = os.path.join(folder, file_name)
            return folder, file
    else:
        file = os.path.join(os.path.normpath(os.getcwd()), file_name)
        return file

def read_ET(file):
    observable = []
    with open(file,"r") as fin:
        for line in fin:
            words = line.split()[1:]
            observable.append(float(words[-2]))
    return np.array(observable)

def prepare_ET_files(some_dict):
    a = []
    
    for i in some_dict:
        a.append(read_ET(some_dict[i]))
        
    return np.stack(a, axis = 1)

def get_atom_arrays(traj_file):
    c1 = []
    c2 = []

    with open(traj_file,"r") as fin:
        for line in fin:
            if line.startswith('C1'):
                words = line.split()[1:]
                c1.append(list(map(float,words)))
            if line.startswith('C2'):
                words = line.split()[1:]
                c2.append(list(map(float,words)))
    return np.array(c1), np.array(c2)

def save_to_csv(name, data, header):
    np.savetxt(name, data.T, delimiter = ';', fmt='%.6e', header = header, comments='')
    
def create_dataframe(data, histogram = True):
    if histogram:
        df_name = pd.read_csv(data, delimiter=";", index_col=0)
        return df_name
    
    else:
        df_name = pd.read_csv(data, delimiter=";")
        df_name.index = df_name.index * 1e-3
        df_name.index.name = 't [ps]'
        return df_name
    
def get_z(traj_file):
    coord = []
    
    with open(traj_file,"r") as fin:
        number = int(fin.readline())
        
        for line in fin:
            if line.startswith('O') or line.startswith('C'):
                words = line.split()[3:]
                coord.append(float(words[0]))
        z = slice_contributions(np.array(coord), number)
    return z

def save_wdd_to_csv(name, data):
    np.savetxt(name, data.T, delimiter = ';', fmt='%.6e', comments='')
    
def save_for_origin(df, name):
    df.to_csv(construct_name(name))

def format_pandas_table(df, caption, caption_style, color = True, vmin=-1, vmax=1, axis=None):
    if color:
        formatted_df = df.style.background_gradient(cmap='seismic', axis = None, vmin = vmin, vmax = vmax)
        formatted_df =  formatted_df.set_caption(caption).set_table_styles(caption_style)
    else:
        formatted_df = df
        formatted_df =  formatted_df.style.set_caption(caption).set_table_styles(caption_style)
    
    return formatted_df

def show_virtual_atoms(vf):
    a = define_fold_or_file(vf, file_name = 'travis.log')
    start_string = 'defined through center'
    end_string = 'tip of molecular dipole vector starting in'
    
    switch = False
    tick = -1
    virt_atoms = {}
    column_name = []
    
    with open(a,"r") as fin:
        for line in fin:
            if start_string in line:
                
                switch = True
                tick += 1
                
                word = " ".join(line.split())
                column_name.append(word)
                dictionary = []
                
            if end_string in line:
                virt_at_df = pd.DataFrame(virt_atoms)
                return virt_at_df
            
            if (switch == True) and (" ".join(line.split()) not in column_name):
                word = " ".join(line.split())
                
                if word == '':
                    pass
                
                else:
                    dictionary.append(word)
                    virt_atoms[column_name[tick]] = dictionary
                    
            else:
                pass

### Calculation
def calculate_difference(v1, v2):
    d = v2 - v1
    return d

def create_normal(c1): # Make surface normal vector
    k = np.mean(c1[:,2])
    n = np.zeros_like(c1)
    n[:,2] = c1[:,2]
    n[:,2] = np.where(c1[:,2]<k, 1, -1)
    return n

def get_length(v):
    r = np.sqrt(v[:,0]**2 + v[:,1]**2 + v[:,2]**2)
    return r

def get_angle(v1, v2):
    m = np.sum(v1*v2, axis = 1)
    
    r1 = get_length(v1)
    r2 = get_length(v2)
    cos_a = m/(r1*r2)
    a = np.degrees(np.arccos(cos_a))
    return a

def slice_contributions(data, number):
    motors = []
    k = 0
    for i in range(number):
        motors.append(data[k::number].copy())
        k += 1
    return np.array(motors)


def calculate_tilt(tilt_fold, system_name):
    (c1, c2) = get_atom_arrays(define_fold_or_file(tilt_fold, file_name = system_name + '_traj_out.xyz'))
    v = calculate_difference(c1, c2)
    n = create_normal(c1)
    angle = get_angle(v, n)
    data = slice_contributions(angle, 8)
    save_to_csv(name = (tilt_fold + '.csv'), data = data, header = 'A;B;C;D;E;F;G;H')

def calculate_confidence_interval(data, confidence = 0.95):
    a = 1.0 * np.array(data)
    n = len(a)
    m, se = np.mean(a),st.sem(a)
    h = se * st.t.ppf((1+confidence) / 2., n-1)
    return [m, m-h, m+h]

def blockAverage(datastream, maxBlockSize=0):
 
    Nobs         = len(datastream)           # total number of observations in datastream
    minBlockSize = 1;                        # min: 1 observation/block
 
    if maxBlockSize == 0:
        maxBlockSize = int(Nobs/4);        # max: 4 blocs (otherwise can't calc variance)
  
    NumBlocks = maxBlockSize - minBlockSize   # total number of block sizes

    blockMean = np.zeros(NumBlocks)               # mean (expect to be "nearly" constant)
    blockVar  = np.zeros(NumBlocks)               # variance associated with each blockSize
    blockCtr  = 0
 
    for blockSize in range(minBlockSize, maxBlockSize):

        Nblock    = int(Nobs/blockSize)               # total number of such blocks in datastream
        obsProp   = np.zeros(Nblock)                  # container for parcelling block 

        # Loop to chop datastream into blocks
        # and take average
        for i in range(1,Nblock+1):
            
            ibeg = (i-1) * blockSize
            iend =  ibeg + blockSize
            obsProp[i-1] = np.mean(datastream[ibeg:iend])

        blockMean[blockCtr] = np.mean(obsProp)
        blockVar[blockCtr]  = np.var(obsProp)/(Nblock - 1)
        blockCtr += 1
 
    v = np.arange(minBlockSize,maxBlockSize)
        
    return v, np.sqrt(blockVar), blockMean # we are returning standard error, not variance

def cummulative_blockAverages(data, bsize=0):
    block_a_dict = {}
    
    for i in list(data):
        temp = np.array(data[i])
        (v, std, blockMean) = blockAverage(temp, maxBlockSize=bsize)
        block_a_dict[i] = pd.DataFrame(np.column_stack([v, std]), columns=['block pair', f'SE({i})'])
        block_a_dict[i].set_index('block pair', inplace=True)
        
    block_a_dict['all'] = pd.concat([block_a_dict[i] for i in block_a_dict], axis = 1)
    return block_a_dict['all']

def calculate_kde(df):
    x = np.linspace(df.values.min()-5, df.values.max()+5, df.iloc[:,1].size)
    y = []
    heading = []
    for i in df.columns:
        density = st.gaussian_kde(df[i])
        y.append(density(x))
        heading.append(str(round(i,1)))

    kde_df = pd.DataFrame(np.array(y).T, index = x, columns = heading)
    return kde_df

def construct_name(name):
    name = plot_fold + '/' + SYSTEM_NAME + '_' + name + '_' 'plot_ready'+ '.csv'
    return name

def calculate_histograms(df):
    y = {}
    heading = []
    x = np.linspace(df.values.min()-1, df.values.max()+1, int(round((df.iloc[:,1].size)/10)))
    x = x.round(1)
    for i in df.columns:
        h, bin_edges = np.histogram(df[i], bins=x)
        y[i] = h
        
    bin_centers = bin_edges[:-1] + np.diff(bin_edges)/2
    hist_df = pd.DataFrame(y, index = bin_centers)
    hist_df.index.name = 'angle [deg]'
    return hist_df

<a name="data_initialisation"></a>
### *Data initialisation*

In [ ]:
%%capture
%load_ext autoreload
%autoreload 2

# Import libraries
import os, os.path
import numpy as np
import scipy.stats as st
from scipy.signal import savgol_filter
import pandas as pd
import pymannkendall as mk
from IPython.display import Image, Video, display, HTML, display_html # enable displaying embedded images

import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Folder of Origin csv files:
plot_fold = define_fold_or_file('plot_fold', file_name = 'None')

# Define trajectory
SYSTEM_NAME = 'trans_azo'
(traj_fold, traj_file) = define_fold_or_file('', file_name = (SYSTEM_NAME + '_traj.xyz'))

# Define where to cut the equilibration part of trajectory, default is 0
TREND_CUTOFF = 0

# Define molecular system variables
cell_v_a = 1622.1 # length of cell vector A in pm
cell_v_b = 1656.5 # length of cell vector B in pm
cell_v_c = 8517.7 # length of cell vector C in pm
cell_alpha = 93.7000 # angle Alpha (between B and C) in deg: [90.0]
cell_beta = 89.8000 # angle Beta (between A and C) in deg: [90.0]
cell_gamma = 60.9300 # angle Gamma (between A and B) in deg: [90.0]

# Define observables to calculate
obs_ind = ['lt', 'ut', 'bend', 'rdf_fh_fh', 'rdf_fh_stand', 
           'wdd_stand', 'wdd_water', 'sdf_stand', 'sdf_fh', 
           'll_cdf_adf', 'll_cdf_rdf', 'ul_cdf_adf', 'ul_cdf_rdf'] 

# Define path to folders with calculated observables
spa_fold = {obs_ind[i] : define_fold_or_file(obs_ind[i], file_name = 'None')
            for i in range(len(obs_ind))}

# Define Travis input files
travis_input = {obs_ind[i] : define_fold_or_file(file_name = ('travis_' + obs_ind[i] + '_input.txt'))
                for i in range(len(obs_ind))}

# Define paths to processed .csv files
csv_file = {obs_ind[i] : define_fold_or_file(file_name = obs_ind[i] + '.csv')
            for i in range(len(obs_ind))}

# Extra .csv file paths should be defined here
csv_file['bend_hist'] = define_fold_or_file(file_name = 'bend_hist' + '.csv')
csv_file['et'] = define_fold_or_file(file_name = 'et' + '.csv')
csv_file['rdf_fh_fh_timedev'] = define_fold_or_file(file_name = 'rdf_fh_fh_timedev' + '.csv')

# Define Energy and Temperature file paths and folder
ET_pref, ET_fold = ['temp', 'etot', 'ekin'] , define_fold_or_file('ET', file_name = 'None')
ET_file = {ET_pref[i] : define_fold_or_file(file_name = (ET_pref[i] + '.dat'))
           for i in range(len(ET_pref))}

# Define color palette
color_scheme = px.colors.qualitative.G10 + ['yellow', 'black','brown','grey']

# Define style of pandas Table captions
caption_style = [{
    'selector': 'caption',
    'props': [('font-size', '10pt'), ('font-weight', 'bold'), ('text-align', 'center')]
                }]

In [ ]:
# Plotting functions
timedev_layout_theme = dict(height=420, barmode='overlay', title_x=0.5, template='simple_white', showlegend=True) #yaxis_tickformat = '.0f',

def multi_table_display(table_list):
    ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
    '''
    return HTML(
        '<table>'+
        ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        '</tr></table>')

def make_simple_histogram(df, name, color, line_width=2, opacity=1):
    hist = go.Histogram(y = df, name=name, legendgroup=name, showlegend=False, opacity = opacity, marker=dict(color=color))
    return hist

def make_simple_line_graph(df, name, color, line_width=2, opacity=1):
    graph = go.Scatter( x=df.index, y=df, name=name, legendgroup=name, mode = 'lines', 
                       opacity = opacity, line = dict(width=line_width, color=color))
    return graph

def make_table(df, headers):
    table = go.Table(header=dict(values=headers, align = "left"),
                     cells=dict(values=[df[k].tolist() for k in df.columns[:]], align = "left"))
    return table

def sort_axes(fig, axes_title):
    fig.update_yaxes(title_text=axes_title, row=1, col=1)
    fig.update_yaxes(title_text=axes_title, row=1, col=2)
    fig.update_xaxes(title_text="<i>occurance</i>", row=1, col=2)
    fig.update_xaxes(title_text='<i>t</i> [ps]', row=1, col=1)

def plot_timedev_obs(df, title, color, obs = 'angle_type', layout = timedev_layout_theme):
    
    axes_title = {'angle_type' : '<i>angle</i> [deg]', 'rdf_type' : '<i>r</i> [A]'}
    
    fig = make_subplots(rows=1, cols=2, column_widths=[0.60, 0.40], specs=[[{"type" : "scatter"},{"type" : "scatter"}]])

    for i,j in enumerate(df):
        fig.add_trace(make_simple_line_graph(df[j], df.columns[i], color[i]), row=1, col=1)
        fig.add_trace(make_simple_histogram(df[j], df.columns[i], color[i], opacity=0.65), row=1, col=2)
     
    sort_axes(fig, axes_title[obs])
    fig.update_layout(layout)
    fig.update_layout(title_text=title)

    return fig

def plot_minimal_distance(df, title, color, layout = timedev_layout_theme):
        
    fig = make_subplots(rows=1, cols=2, column_widths=[0.60, 0.40], specs=[[{"type" : "scatter"},{"type" : "scatter"}]])

    for i,j in enumerate(df):
        if i == 0:
            fig.add_trace(make_simple_line_graph(df[j], j, 'red', line_width=3, opacity=1),row=1,col=1)
            fig.add_trace(make_simple_histogram(df[j], j, 'red', line_width=3, opacity=1),row=1,col=2)
            
        else:
            fig.add_trace(make_simple_line_graph(df[j], df.columns[i], 'orange', line_width=2, opacity=0.65), row=1, col=1)
            fig.add_trace(make_simple_histogram(df[j], df.columns[i], 'orange', line_width=2, opacity=0.65), row=1, col=2)
    
    sort_axes(fig, "<i>r</i> [A]")
    fig.update_layout(layout)
    fig.update_layout(title_text=title)

    return fig

def plot_ci_and_mean(df1, df2, title, color, layout = timedev_layout_theme):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(['General statistics for ' + title,
                                                         'Temporal evolution of mean ' + title + ' angle, with 95 % confidence intervals']), 
                        column_widths=[0.4, 0.6], shared_xaxes=False, 
                        specs=[[{"type": "table"}, {"type" : "scatter"}]])
                       
    fig.add_trace(go.Table(make_table(df1, df1.columns)),row=1, col=1)

    fig.add_trace(go.Scatter(x=df2.index, y=df2.iloc[:,1], fillcolor=color[2], line_color=color[2], opacity = 0.65, 
                             showlegend=True, legendgroup='ci', name='<i>95% confidence interval</i>'), row=1, col=2)

    fig.add_trace(go.Scatter(x=df2.index, y=df2.iloc[:,2], fill='tonexty', line_color=color[2], opacity = 0.65, 
                             showlegend=False, legendgroup='ci', name='<i>95% confidence interval</i>'), row=1, col=2)

    fig.add_trace(go.Scatter(x=df2.index, y=df2.iloc[:,0], line_color=color[1], name='Mean'), row=1, col=2)
    
    fig.update_yaxes(title_text="<i>angle</i> [deg]", row=1, col=2)
    fig.update_yaxes(title_text="<i>angle</i> [deg]", row=1, col=2)
    fig.update_xaxes(title_text='<i>t</i> [ps]', row=1, col=2)
    fig.update_layout(layout)
    return fig

def plot_MK_and_block_averages(df1, df2, title, color, layout = timedev_layout_theme):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(['Block averaging', 'Mann Kendall statistics']), 
                        column_widths=[0.5, 0.5], shared_xaxes=False, 
                        specs=[[{"type": "scatter"}, {"type" : "table"}]])

    fig.add_trace(go.Table(make_table(df2, df2.columns)), row=1, col=2)

    for i, j in enumerate(df1):
        fig.add_trace(make_simple_line_graph(df1[j], df1.columns[i], color[i]), row=1, col=1)

    fig.update_yaxes(title_text="<i>SE</i> [deg]", row=1, col=1)
    fig.update_xaxes(title_text='<i>block pair size</i>', row=1, col=1)

    fig.update_layout(layout)
    fig.update_layout(title_text = 'Error and trendlessness estimate for ' + title, 
                      height = 420, width = 1240, legend=dict(yanchor="top", y=0.99, xanchor="left", x=0.36))
        
    return fig

def plot_et_data(df1, df2, title, color, layout=timedev_layout_theme):
    
    values=['index',
            '<i>T</i> [K]',
            '10<sup>-6</sup> <i>E</i><sub>TOT</sub> [kJmol<sup>-1</sup>]',
            '10<sup>-6</sup> <i>E</i><sub>KIN</sub> [kJmol<sup>-1</sup>]']
    
    fig = make_subplots(rows=3, cols=2,
                        column_widths=[0.6, 0.4],
                        row_heights=[0.3, 0.3, 0.3],
                        horizontal_spacing=0.15,
                        vertical_spacing=0.0,
                        shared_xaxes=True,
                        specs=[[{"type" : "scatter"}, {"type": "table", "rowspan": 3}],
                               [{"type" : "scatter"},             None               ], 
                               [{"type" : "scatter"},             None               ]])
    
    fig.add_trace(go.Table(make_table(df1, values)), row=1, col=2)
    
    for i,j in enumerate(df2):
        k = i+1
        fig.add_trace(make_simple_line_graph(df2[j], values[k], color[i]), row=k, col=1)
        fig.update_yaxes(title_text=values[k], row=(k), col=1)
        fig.update_xaxes(title_text='<i>t</i> [ps]', row=(k), col=1)
        
    fig.update_layout(layout)
    fig.update_layout(title_text='Thermodynamics of <b>'+title+'</b> system', height=640, showlegend = False, yaxis_tickformat = '.1f')

    return fig

def plot_savgol_plot(df1, df2, color, layout = timedev_layout_theme):
    
    fig = make_subplots(rows=1, cols=2,
                        subplot_titles=(['Total energy MK statistics','Savitzky–Golay filter of total energy']),
                        column_widths=[0.30, 0.70],
                        specs=[[{"type": "table"}, {"type": "scatter"}]],)
    
    fig.add_trace(make_table(df1, df1.columns), row=1, col=1)
                        
    for i, j in enumerate(df2.columns):
        fig.add_trace(go.Scatter(
        x=df2.index,
        y=df2[j],
        line_color=color[i],
        line = dict(width = 1+(i*0.25)),
        opacity = np.log(i+2.5)/np.log(len(df2.columns)+1.5),
        name=j,
        ),row=1,col=2)
    
    fig.update_layout(layout)
    
    return fig

def plot_water_dist(df1, df2, color, layout = timedev_layout_theme):

    fig = go.Figure()
    legend_switch = True
    
    for i, j in enumerate(df1.columns):
                fig.add_trace(go.Scatter(
                x=df1.index,
                y=df1[j],
                line_color=color[1],
                line = dict(width = 2),
                opacity = 0.45,
                showlegend=legend_switch,
                legendgroup='water',
                name='<i>z</i>(water)'))
            
                legend_switch = False
                
    legend_switch = True
    for i, j in enumerate(df2.columns):
                fig.add_trace(go.Scatter(
                x=df2.index,
                y=df2[j],
                line_color=color[0],
                line = dict(width = 2),
                showlegend=legend_switch,
                legendgroup='motors',
                name='<i>z</i>(triptycene stand)'))
            
                legend_switch = False

    fig.update_yaxes(title_text="<i>d</i><sub>z</sub> [A]")
    fig.update_xaxes(title_text='<i>t</i> [ps]')
    
    fig.update_layout(layout)
    fig.update_layout(title_text='Water vs tryptycene stand density distribution', hovermode = False, height=440)
    
    return fig

def plot_cdf_type_data(df1, df2, title, layout = timedev_layout_theme):

    fig = make_subplots(rows=1, cols=2, 
                        subplot_titles=(['Combined distribution function', 'Correlation matrix']),
                        column_widths=[0.5, 0.5],
                        shared_xaxes=False,
                        specs=[[{"type": "Histogram2d"}, 
                               {"type" : "Histogram2d"}]])
    
    fig.add_trace(go.Histogram2d(
        x = df1.iloc[:,0],
        y = df1.iloc[:,1],
        z = df1.iloc[:,2],
        histfunc="avg",
        colorscale = 'Jet',
        nbinsx=100,
        nbinsy=100,
        colorbar = dict(orientation = 'h',
                len = 0.22,
                 x = 0.35,
                 y = -0.45,),
        ), row=1, col=1)
    
    fig.add_trace(go.Histogram2d(
        x = df2.iloc[:,0],
        y = df2.iloc[:,1],
        z = df2.iloc[:,2],
        histfunc="avg",
        colorscale = 'Jet',
        nbinsx=100,
        nbinsy=100,

        colorbar = dict(orientation = 'h',
                        len = 0.22,
                         x = 0.90,
                         y = -0.45,),
        ), row=1, col=2)

    fig.update_coloraxes = dict(cmid = 0, cauto = "True", row=1, col=2)
    fig.update_layout(layout)
    fig.update_layout(title_text=title, height=440, showlegend=False, hovermode = False)


    return fig

<a name="trajectory_preprocessing"></a>
## **Trajectory preprocessing**
***

In [ ]:
# Create all necessary directories (run only once)
for i in spa_fold:
    if not os.path.isdir(spa_fold[i]):
        bspa_fold = spa_fold[i]
        ! mkdir $bspa_fold
    else:
        pass

# Make origin plot folder
if not os.path.isdir(plot_fold):
    ! mkdir plot_fold;
    
# Make energy directory
if not os.path.isdir(ET_fold):
    ! mkdir ET;

video_fold = define_fold_or_file('md_video', file_name = 'None')

if not os.path.isfile(video_fold + '/travis.log'):
    make_md_video = define_fold_or_file(file_name = 'make_md_video.sh')
    ! bash $make_md_video;

<a name="display_topology"></a>
### *Display topology*

In [ ]:
display(show_virtual_atoms(video_fold))
topo_img = (video_fold + '/img.png')
display(Image(topo_img, height = 420, width = 640))

<a name="video"></a>
## **Trajectory visualisation**
***

In [ ]:
### Upload video

<a name="calculation"></a>
## **Calculation**

In [ ]:
%%capture
# RDFs should be run first, as they require unwrapped no-PBC trajectories
if not os.path.isfile(spa_fold['rdf_fh_fh'] + '/travis.log'):
    proc_rdf_fh_fh = define_fold_or_file(file_name = 'proc_rdf_fh_fh.sh')
    ! bash $proc_rdf_fh_fh;

if not os.path.isfile(spa_fold['rdf_fh_stand'] + '/travis.log'):
    proc_rdf_fh_stand = define_fold_or_file(file_name = 'proc_rdf_fh_stand.sh')
    ! bash $proc_rdf_fh_stand;

# run Travis analysis and clear afterwards
for i in spa_fold:
    bspa_fold = spa_fold[i]
    btravis_input = travis_input[i]
    if not os.path.isfile(bspa_fold + '/travis.log'):
        ! cd $bspa_fold; travis -p $traj_file -i $btravis_input > travis.out
        
        if not os.path.isfile(bspa_fold + '/img.png'):
            ! cd $bspa_fold; rm ./*.dot ./*no_H.png ./*H2O.png; mv ./*.png ./img.png
        else:
            ! cd $bspa_fold; mv ./img.png ./tmp; rm ./*.dot ./*.png; mv ./tmp ./img.png

In [ ]:
%%capture
# rename .csv RDF timedev data
! bash rename_ugly.sh

# Change units of energy from [eV] to kJmol-1
eV = 96.48530749925793 # kJmol-1
et_arr = prepare_ET_files(ET_file)
et_arr_ev = et_arr.copy()
et_arr_ev[:,1:3] = et_arr[:,1:3]*eV

# Reformat ET files to match others
save_to_csv(csv_file['et'], et_arr_ev.T, header = 'T [K];Etot [kJmol-1];Ekin [kJmol-1]') # Function transponses it automatically, 
                                                                                         # so it needs to be transposed again
# Rename and reframe Travis outputs where necessary
if os.path.isfile(spa_fold['bend'] + '/bend_timedev.csv'):
    # print('Travis processed adf .csv data already renamed!')
    pass
else:
    bspa_fold = spa_fold['bend']
    ! cd $bspa_fold; mv ./adf_timedev* ./bend_timedev.csv; mv ./adf_*.csv ./bend_hist.csv

# reformat bending files to match the rest - reformat time development file
bend_npa = np.loadtxt(spa_fold['bend'] + '/bend_timedev.csv', delimiter = ';', 
                      unpack = True, usecols = tuple(range(1, 8 + 1)))
save_to_csv(csv_file['bend'], bend_npa, 'A;B;C;D;E;F;G;H')

# reformat bending files to match the rest - reformat histogram file
bend_hist_npa = np.loadtxt(spa_fold['bend'] + '/bend_hist.csv', delimiter = ';', 
                      unpack = True, skiprows = 1)
save_to_csv(csv_file['bend_hist'], bend_hist_npa, header = 'angle [deg];occurance') # maybe add a hist dictionary later


# Create csv files for water and stand z-axis density distribution
# Take the z component density of oxygen atoms at every time step
z_o = get_z(define_fold_or_file(spa_fold['wdd_water'], file_name = SYSTEM_NAME + '_traj_out.xyz'))
z_stand = get_z(define_fold_or_file(spa_fold['wdd_stand'], file_name = SYSTEM_NAME + '_traj_out.xyz'))

# Save water and stand distributions to .csv
save_wdd_to_csv(name = csv_file['wdd_water'], data = z_o)
save_wdd_to_csv(name = csv_file['wdd_stand'], data = z_stand)

# calculate lower and upper tilts and save the files in .csv format
calculate_tilt(spa_fold['lt'], SYSTEM_NAME)
calculate_tilt(spa_fold['ut'], SYSTEM_NAME)

<a name="dataframe_creation"></a>
### *DataFrame creation*

In [ ]:
df_dict = {}
for i in (csv_file.keys()):
    if os.path.isfile(csv_file[i]):
        if not ('_hist' in i):
            # print(f"{i} DataFrame successfuly created!")
            df_dict[i] = create_dataframe(csv_file[i], histogram = False)
        else:
            # print(f"{i} DataFrame successfuly created!")
            df_dict[i] = create_dataframe(csv_file[i], histogram = True)
    else:
        # print(f"{i}.csv does not exist, impossible to create {i} DataFrame...") # use this for error checking. Everything that needs to be represented
        pass                                                                        # as a DataFrame should become a dataFrame by now

# Read csv file to a dataframe
df_dict['wdd_water'] = pd.read_csv(csv_file['wdd_water'], delimiter=';', header = None)
df_dict['wdd_stand'] = pd.read_csv(csv_file['wdd_stand'], delimiter=';', header = None)

<a name="thermodynamics"></a>
## **Thermodynamics**
***

In [ ]:
# Create energy and temperature dataFrame
PROPERTY_NAME = 'thermo'
df = df_dict['et']

# take every hundreth data point (time step is now 100 fs)
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df2.loc[:,'Etot [10^-6 kJmol-1]'] = df2.loc[:,'Etot [kJmol-1]']*(1e-6)
df2.loc[:,'Ekin [10^-3 kJmol-1]'] = df2.loc[:,'Ekin [kJmol-1]']*(1e-3)
df2 = df2.drop(columns=['Etot [kJmol-1]', 'Ekin [kJmol-1]'])
df2 = df2.round(5)

# CORRECT TEMPERATURE
df2['T [K]'] = df2['T [K]'] - 13

# Calculate and save general statistics DataFrame
stat_df = df2.describe().drop('count').round(4)
save_for_origin(stat_df, PROPERTY_NAME+'_stat')
save_for_origin(df2, PROPERTY_NAME)

stat_df.insert(0, 'index', stat_df.index)
df2.loc[:,'T [K]'] = df2.loc[:,'T [K]'].round(1)
fig = plot_et_data(stat_df, df2, SYSTEM_NAME, color_scheme)
fig.show()

<a name="equilibration"></a>
### *Equilibration length estimate*

In [ ]:
# Perform Savitsky-Golay smooting
savgol_window_sizes = [21, 51, 101, 201, 401, 801]

etot_savgol_df = pd.DataFrame({'ws: '+str(i) : savgol_filter(df2['Etot [10^-6 kJmol-1]'], i, 3) for i in savgol_window_sizes})
etot_savgol_df.index = df2.index.copy()
extracted_col = df2['Etot [10^-6 kJmol-1]']
etot_savgol_df.insert(0, 'ws: 0', extracted_col)

# Mann Kendall on whole trajectory
etot_mk = []
time_window = []

tick_start = 0
tick = int(round(len(df2)-(TREND_CUTOFF*10))/7)
tick_end = len(df2)-tick

for i in range(0, tick_end, tick): # choose interval chunks #int(round(len(df2)/2))
    etot_mk.append(mk.hamed_rao_modification_test(df2.iloc[i:(i+tick),1], alpha = 0.01)) #hamed_rao_modification_test
    time_window.append(str(round((i*0.1)))+'-'+str(round((i+tick)*0.1)))

etot_mk_pd = pd.DataFrame(etot_mk)
etot_mk_pd.index = time_window
etot_mk_pd.index.name = 'time window [ps]'
etot_mk_pd = (etot_mk_pd.drop(['h','var_s', 'Tau'], axis = 1)).round(4).copy() #.copy()


save_for_origin(etot_savgol_df, 'Savitsky-Golay_Etot')
fig_ = plot_savgol_plot(etot_mk_pd,etot_savgol_df, color_scheme)

fig_.update_yaxes(title_text="'10<sup>-6</sup> <i>E</i><sub>TOT</sub> [kJmol<sup>-1</sup>]'")
fig_.update_xaxes(title_text='<i>t</i> [ps]')

display(fig_)

In [ ]:
TREND_CUTOFF = 200

<a name="MK"></a>
### *Mann-Kendall statistics*

In [ ]:
tick_start = TREND_CUTOFF*10
tick = int(round(len(df2)-(TREND_CUTOFF*10))/5)
tick_end = len(df2)-tick

[temp_mk_pd, etot_mk_pd, ekin_mk_pd] = 0, 0, 0
thermo_df = [temp_mk_pd, etot_mk_pd, ekin_mk_pd]

caption = ['Temperature', 'Total_energy', 'Kinetic_energy']

for k in range(0,3):
    temp_mk = []
    time_window  = []

    for i in range(tick_start, tick_end, tick): # choose interval chunks by changing denominator in tick variable
        temp_mk.append(mk.hamed_rao_modification_test(df2.iloc[i:(i+tick),k], alpha = 0.01))
        time_window.append(str(round((i*0.1)))+'-'+str(round((i+tick)*0.1)))
        
    thermo_df[k] = pd.DataFrame(temp_mk)
    thermo_df[k].index = time_window
    thermo_df[k].index.name = 'time window [ps]'
    thermo_df[k] = (thermo_df[k]).drop(['h','var_s', 'Tau'], axis = 1) #.copy()
    
    save_for_origin(thermo_df[k], caption[k] + ' MK_statistics')

    thermo_df[k] = thermo_df[k].style.set_caption( caption[k] + ' MK_statistics').set_table_styles(caption_style)

display(multi_table_display(thermo_df))

<a name="water"></a>
## **Water distribution density**
***

In [ ]:
# Reformat dataframes
PROPERTY_NAME = 'z_distribution'

df_o = df_dict['wdd_water'].copy()
df_stand = df_dict['wdd_stand'].copy()

df_o.index = df_o.index * 1e-3
df_o.index.name = 't [ps]'
df_o2 = df_o[(np.arange(len(df_o)) % 100 == 0)].copy()
df_o2.index = df_o2.index * 10
df_o2 = df_o2.drop(df_o2.index[:TREND_CUTOFF])
df_o2.columns = ['h2o_'+str(i) for i in range(0, len(df_o2.columns))]

df_stand.index = df_stand.index * 1e-3
df_stand.index.name = 't [ps]'
df_stand2 = df_stand[(np.arange(len(df_stand)) % 100 == 0)].copy()
df_stand2.index = df_stand2.index * 10
df_stand2 = df_stand2.drop(df_stand2.index[:TREND_CUTOFF])
df_stand2.columns = list('ABCDEFGH')

fig = plot_water_dist(df_o2, df_stand2, color_scheme)
fig.show()

save_for_origin(df_stand2, PROPERTY_NAME+'_stand_timedev')
save_for_origin(df_o2, PROPERTY_NAME+'_water_timedev')

<a name="lower_tilt"></a>
## **Triptycene stand tilt**
***

In [ ]:
PROPERTY_NAME = 'lower_tilt'

df = df_dict['lt']
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df3 = df2.round(2)
df3.index = df3.index * 10 # we took every hundreth step, we have to scale the abscissa
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Plot data
fig = plot_timedev_obs(df3, 'Temporal evolution of ' + PROPERTY_NAME + ' angle for <b>' + SYSTEM_NAME + '</b> system', color_scheme, obs = 'angle_type')
display(fig)

### *General statistics - metastable layer (EFGH)*

In [ ]:
PROPERTY_NAME = 'EFGH_lower_tilt'

df = (df_dict['lt'])[['E','F','G','H']]
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()
df3 = df2.copy()

# Calculate confidence intervals on all rows. (we cannot estimate whether we are working with normal distribution here, as per calculation, our sample size is just 8)
df3[['mean', 'lci', 'uci']] = df2.apply(lambda row : calculate_confidence_interval(row), axis = 1, result_type='expand')
df3 = df3.round(4)
df3.index = df3.index * 10 # position of this line is important, it should be put after the calculation and smoothing has taken place
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Calculate cummulative block averages
df4 = pd.DataFrame(cummulative_blockAverages(df3.iloc[:,0:9], bsize = 150))

# Perform Mann Kendall test
mann_kendall = {}
for i in df3.iloc[:,0:9]:
    mann_kendall[i] = mk.hamed_rao_modification_test(df3[i], alpha = 0.05)

df5 = pd.DataFrame(mann_kendall).T
df5.columns = ['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']
df5.insert(0, 'obs', df5.index.to_numpy())
df5[['p', 'z', 'Tau', 'slope',]] = df5[['p', 'z', 'Tau', 'slope']].astype(float).round(4)

#Save time development
save_for_origin(df3, PROPERTY_NAME+'_timedev')

#Construct and save histograms
hist_df= calculate_histograms(df3.drop(['uci','lci'], axis = 1))
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save block averages
save_for_origin(df4.drop(['SE(lci)', 'SE(uci)'], axis = 1), PROPERTY_NAME+'_block_averages')

# Save Mann Kendall table
save_for_origin(df5.drop(['obs', 'h'], axis = 1), PROPERTY_NAME+'_MK')

# Calculate and save general statistics DataFrame
stat_df = df3.describe().drop('count').drop(['lci', 'uci'], axis = 1).round(2) #.drop( 'lci', 'uci')
save_for_origin(stat_df, PROPERTY_NAME+'_stat')

stat_df2 = stat_df.copy()
stat_df2.insert(0, 'index', stat_df.index)

# Plot the results
fig1 = plot_ci_and_mean(stat_df2, df3[['mean', 'lci', 'uci']], PROPERTY_NAME, color_scheme)
fig1.show()

# Plot results:
fig2 =  plot_MK_and_block_averages(df4, df5[['obs','trend', 'p', 'z','slope']], PROPERTY_NAME, color_scheme)
fig2.show()

### *General statistics - unstable layer (ABCD)*

In [ ]:
PROPERTY_NAME = 'ABCD_lower_tilt'

df = (df_dict['lt'])[['A','B','C','D']]
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()
df3 = df2.copy()

# Calculate confidence intervals on all rows. (we cannot estimate whether we are working with normal distribution here, as per calculation, our sample size is just 8)
df3[['mean', 'lci', 'uci']] = df2.apply(lambda row : calculate_confidence_interval(row), axis = 1, result_type='expand')
df3 = df3.round(4)
df3.index = df3.index * 10 # position of this line is important, it should be put after the calculation and smoothing has taken place
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Calculate cummulative block averages
df4 = pd.DataFrame(cummulative_blockAverages(df3.iloc[:,0:9], bsize = 340))

# Perform Mann Kendall test
mann_kendall = {}
for i in df3.iloc[:,0:9]:
    mann_kendall[i] = mk.hamed_rao_modification_test(df3[i], alpha = 0.05)

df5 = pd.DataFrame(mann_kendall).T
df5.columns = ['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']
df5.insert(0, 'obs', df5.index.to_numpy())
df5[['p', 'z', 'Tau', 'slope',]] = df5[['p', 'z', 'Tau', 'slope']].astype(float).round(4)

#Save time development
save_for_origin(df3, PROPERTY_NAME+'_timedev')

#Construct and save histograms
hist_df= calculate_histograms(df3.drop(['uci','lci'], axis = 1))
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save block averages
save_for_origin(df4.drop(['SE(lci)', 'SE(uci)'], axis = 1), PROPERTY_NAME+'_block_averages')

# Save Mann Kendall table
save_for_origin(df5.drop(['obs', 'h'], axis = 1), PROPERTY_NAME+'_MK')

# Calculate and save general statistics DataFrame
stat_df = df3.describe().drop('count').drop(['lci', 'uci'], axis = 1).round(2) #.drop( 'lci', 'uci')
save_for_origin(stat_df, PROPERTY_NAME+'_stat')

stat_df2 = stat_df.copy()
stat_df2.insert(0, 'index', stat_df.index)

# Plot the results
fig1 = plot_ci_and_mean(stat_df2, df3[['mean', 'lci', 'uci']], PROPERTY_NAME, color_scheme)
fig1.show()

# Plot results:
fig2 =  plot_MK_and_block_averages(df4, df5[['obs','trend', 'p', 'z','slope']], PROPERTY_NAME, color_scheme)
fig2.show()

### *Correlation matrices*

In [ ]:
PROPERTY_NAME = 'lower_tilt'

df = df_dict['lt']
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df3 = df2.round(2)
df3.index = df3.index * 10 # we took every hundreth step, we have to scale the abscissa
df3 = df3.drop(df3.index[:TREND_CUTOFF])

corr_type = ['pearson','kendall','spearman']
corr_type_df = {i : '' for i in corr_type}

for i in corr_type:
    
    corr_type_df[i] = df3.corr(i)
    
    #Save correlation plots
    save_for_origin(corr_type_df[i], PROPERTY_NAME+'_'+i+'_corr_matrix')
    
    # Format pandas table for display
    corr_type_df[i] = format_pandas_table(corr_type_df[i], i +' type correlation matrix', caption_style)

multi_table_display([i for i in corr_type_df.values()])

<a name="upper_tilt"></a>
## **Functional head tilt**
***

In [ ]:
PROPERTY_NAME = 'upper_tilt'

df = df_dict['ut']
df2 = df[(np.arange(len(df)) % 10 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df3 = df2.round(2)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:(TREND_CUTOFF*10)])

# Plot data
fig = plot_timedev_obs(df3, 'Temporal evolution of ' + PROPERTY_NAME + ' angle for <b>' + SYSTEM_NAME + '</b> system', color_scheme, obs = 'angle_type')
display(fig)

### *General statistics - metastable layer (EFGH)*

In [ ]:
PROPERTY_NAME = 'EFGH_upper_tilt'

df = (df_dict['ut'])[['E','F','G','H']]
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()
df3 = df2.copy()

# Calculate confidence intervals on all rows. (we cannot estimate whether we are working with normal distribution here, as per calculation, our sample size is just 8)
df3[['mean', 'lci', 'uci']] = df2.apply(lambda row : calculate_confidence_interval(row), axis = 1, result_type='expand')
df3 = df3.round(4)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Calculate cummulative block averages
df4 = pd.DataFrame(cummulative_blockAverages(df3.iloc[:,0:9], bsize = 200))

# perform Mann Kendall test
mann_kendall = {}
for i in df3.iloc[:,0:9]:
    mann_kendall[i] = mk.hamed_rao_modification_test(df3[i], alpha = 0.05)

df5 = pd.DataFrame(mann_kendall).T
df5.columns = ['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']
df5.insert(0, 'obs', df5.index.to_numpy())
df5[['p', 'z', 'Tau', 'slope',]] = df5[['p', 'z', 'Tau', 'slope']].astype(float).round(4)

#Save time development
save_for_origin(df3, PROPERTY_NAME+'_timedev')

#Construct and save histograms
hist_df= calculate_histograms(df3.drop(['uci','lci'], axis = 1))
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save block averages
save_for_origin(df4.drop(['SE(lci)', 'SE(uci)'], axis = 1), PROPERTY_NAME+'_block_averages')

# Save Mann Kendall table
save_for_origin(df5.drop(['obs', 'h'], axis = 1), PROPERTY_NAME+'_MK')

# Calculate and save general statistics DataFrame
stat_df = df3.describe().drop('count').drop(['lci', 'uci'], axis = 1).round(2) #.drop( 'lci', 'uci')
save_for_origin(stat_df, PROPERTY_NAME+'_stat')

stat_df2 = stat_df.copy()
stat_df2.insert(0, 'index', stat_df.index)

# Plot the results
fig1 = plot_ci_and_mean(stat_df2, df3[['mean', 'lci', 'uci']], PROPERTY_NAME, color_scheme)
fig1.show()

# Plot results:
fig2 =  plot_MK_and_block_averages(df4, df5[['obs','trend', 'p', 'z','slope']], PROPERTY_NAME, color_scheme)
fig2.show()

### *General statistics - unstable layer (ABCD)*

In [ ]:
PROPERTY_NAME = 'ABCD_upper_tilt'

df = (df_dict['ut'])[['A','B','C','D']]
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()
df3 = df2.copy()

# Calculate confidence intervals on all rows. (we cannot estimate whether we are working with normal distribution here, as per calculation, our sample size is just 8)
df3[['mean', 'lci', 'uci']] = df2.apply(lambda row : calculate_confidence_interval(row), axis = 1, result_type='expand')
df3 = df3.round(4)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Calculate cummulative block averages
df4 = pd.DataFrame(cummulative_blockAverages(df3.iloc[:,0:9], bsize = 200))

# perform Mann Kendall test
mann_kendall = {}
for i in df3.iloc[:,0:9]:
    mann_kendall[i] = mk.hamed_rao_modification_test(df3[i], alpha = 0.05)

df5 = pd.DataFrame(mann_kendall).T
df5.columns = ['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']
df5.insert(0, 'obs', df5.index.to_numpy())
df5[['p', 'z', 'Tau', 'slope',]] = df5[['p', 'z', 'Tau', 'slope']].astype(float).round(4)

#Save time development
save_for_origin(df3, PROPERTY_NAME+'_timedev')

#Construct and save histograms
hist_df= calculate_histograms(df3.drop(['uci','lci'], axis = 1))
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save block averages
save_for_origin(df4.drop(['SE(lci)', 'SE(uci)'], axis = 1), PROPERTY_NAME+'_block_averages')

# Save Mann Kendall table
save_for_origin(df5.drop(['obs', 'h'], axis = 1), PROPERTY_NAME+'_MK')

# Calculate and save general statistics DataFrame
stat_df = df3.describe().drop('count').drop(['lci', 'uci'], axis = 1).round(2) #.drop( 'lci', 'uci')
save_for_origin(stat_df, PROPERTY_NAME+'_stat')

stat_df2 = stat_df.copy()
stat_df2.insert(0, 'index', stat_df.index)

# Plot the results
fig1 = plot_ci_and_mean(stat_df2, df3[['mean', 'lci', 'uci']], PROPERTY_NAME, color_scheme)
fig1.show()

# Plot results:
fig2 =  plot_MK_and_block_averages(df4, df5[['obs','trend', 'p', 'z','slope']], PROPERTY_NAME, color_scheme)
fig2.show()

### *Correlation matrices*

In [ ]:
PROPERTY_NAME = 'upper_tilt'

df = df_dict['ut']
df2 = df[(np.arange(len(df)) % 10 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df3 = df2.round(2)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:(TREND_CUTOFF*10)])

corr_type = ['pearson','kendall','spearman']
corr_type_df = {i : '' for i in corr_type}

for i in corr_type:
    
    corr_type_df[i] = df3.corr(i)
    
    #Save correlation plots
    save_for_origin(corr_type_df[i], PROPERTY_NAME+'_'+i+'_corr_matrix')
    
    # Format pandas table for display
    corr_type_df[i] = format_pandas_table(corr_type_df[i], i +' type correlation matrix', caption_style)

multi_table_display([i for i in corr_type_df.values()])

<a name="bend"></a>
## **Functional head bend**
***

In [ ]:
PROPERTY_NAME = 'bend'

df = df_dict['bend']
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df3 = df2.round(2)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Plot data
fig = plot_timedev_obs(df3, 'Temporal evolution of ' + PROPERTY_NAME + ' angle for <b>' + SYSTEM_NAME + '</b> system', color_scheme, obs = 'angle_type')
display(fig)

### *General statistics - metastable layer (EFGH)*

In [ ]:
PROPERTY_NAME = 'EFGH_bend'
df = (df_dict['bend'])[['E','F','G','H']]
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()
df3 = df2.copy()

# Calculate confidence intervals on all rows. (we cannot estimate whether we are working with normal distribution here, as per calculation, our sample size is just 8)
df3[['mean', 'lci', 'uci']] = df2.apply(lambda row : calculate_confidence_interval(row), axis = 1, result_type='expand')
df3 = df3.round(4)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Calculate cummulative block averages
df4 = pd.DataFrame(cummulative_blockAverages(df3.iloc[:,0:9], bsize = 200))

# perform Mann Kendall test
mann_kendall = {}
for i in df3.iloc[:,0:9]:
    mann_kendall[i] = mk.hamed_rao_modification_test(df3[i], alpha = 0.05)

df5 = pd.DataFrame(mann_kendall).T
df5.columns = ['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']
df5.insert(0, 'obs', df5.index.to_numpy())
df5[['p', 'z', 'Tau', 'slope',]] = df5[['p', 'z', 'Tau', 'slope']].astype(float).round(4)

#Save time development
save_for_origin(df3, PROPERTY_NAME+'_timedev')

#Construct and save histograms
hist_df= calculate_histograms(df3.drop(['uci','lci'], axis = 1))
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save block averages
save_for_origin(df4.drop(['SE(lci)', 'SE(uci)'], axis = 1), PROPERTY_NAME+'_block_averages')

# Save Mann Kendall table
save_for_origin(df5.drop(['obs', 'h'], axis = 1), PROPERTY_NAME+'_MK')

# Calculate and save general statistics DataFrame
stat_df = df3.describe().drop('count').drop(['lci', 'uci'], axis = 1).round(2) #.drop( 'lci', 'uci')
save_for_origin(stat_df, PROPERTY_NAME+'_stat')

stat_df2 = stat_df.copy()
stat_df2.insert(0, 'index', stat_df.index)

# Plot the results
fig1 = plot_ci_and_mean(stat_df2, df3[['mean', 'lci', 'uci']], PROPERTY_NAME, color_scheme)
fig1.show()

# Plot results:
fig2 =  plot_MK_and_block_averages(df4, df5[['obs','trend', 'p', 'z','slope']], PROPERTY_NAME, color_scheme)
fig2.show()

### *General statistics - metastable layer (ABCD)*

In [ ]:
PROPERTY_NAME = 'ABCD_bend'
df = (df_dict['bend'])[['A','B','C','D']]
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()
df3 = df2.copy()

# Calculate confidence intervals on all rows. (we cannot estimate whether we are working with normal distribution here, as per calculation, our sample size is just 8)
df3[['mean', 'lci', 'uci']] = df2.apply(lambda row : calculate_confidence_interval(row), axis = 1, result_type='expand')
df3 = df3.round(4)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:TREND_CUTOFF])

# Calculate cummulative block averages
df4 = pd.DataFrame(cummulative_blockAverages(df3.iloc[:,0:9], bsize = 320))

# perform Mann Kendall test
mann_kendall = {}
for i in df3.iloc[:,0:9]:
    mann_kendall[i] = mk.hamed_rao_modification_test(df3[i], alpha = 0.05)

df5 = pd.DataFrame(mann_kendall).T
df5.columns = ['trend', 'h', 'p', 'z', 'Tau', 's', 'var_s', 'slope', 'intercept']
df5.insert(0, 'obs', df5.index.to_numpy())
df5[['p', 'z', 'Tau', 'slope',]] = df5[['p', 'z', 'Tau', 'slope']].astype(float).round(4)

#Save time development
save_for_origin(df3, PROPERTY_NAME+'_timedev')

#Construct and save histograms
hist_df= calculate_histograms(df3.drop(['uci','lci'], axis = 1))
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save block averages
save_for_origin(df4.drop(['SE(lci)', 'SE(uci)'], axis = 1), PROPERTY_NAME+'_block_averages')

# Save Mann Kendall table
save_for_origin(df5.drop(['obs', 'h'], axis = 1), PROPERTY_NAME+'_MK')

# Calculate and save general statistics DataFrame
stat_df = df3.describe().drop('count').drop(['lci', 'uci'], axis = 1).round(2) #.drop( 'lci', 'uci')
save_for_origin(stat_df, PROPERTY_NAME+'_stat')

stat_df2 = stat_df.copy()
stat_df2.insert(0, 'index', stat_df.index)

# Plot the results
fig1 = plot_ci_and_mean(stat_df2, df3[['mean', 'lci', 'uci']], PROPERTY_NAME, color_scheme)
fig1.show()

# Plot results:
fig2 =  plot_MK_and_block_averages(df4, df5[['obs','trend', 'p', 'z','slope']], PROPERTY_NAME, color_scheme)
fig2.show()

### *Correlation matrices*

In [ ]:
PROPERTY_NAME = 'bend'

df = df_dict['bend']
df2 = df[(np.arange(len(df)) % 100 == 0)].copy()  # len of multicolumn DataFrame returns number of rows
df3 = df2.round(2)
df3.index = df3.index * 10
df3 = df3.drop(df3.index[:TREND_CUTOFF])

corr_type = ['pearson','kendall','spearman']
corr_type_df = {i : '' for i in corr_type}

for i in corr_type:
    
    corr_type_df[i] = df3.corr(i)
    
    #Save correlation plots
    save_for_origin(corr_type_df[i], PROPERTY_NAME+'_'+i+'_corr_matrix')
    
    # Format pandas table for display
    corr_type_df[i] = format_pandas_table(corr_type_df[i], i +' type correlation matrix', caption_style)

multi_table_display([i for i in corr_type_df.values()])

<a name="upper_layer"></a>
## **Upper layer interactions**
***
### *Functional head - triptycene stand distances*

In [ ]:
PROPERTY_NAME = 'ABCD_fh_stand_rdf'
fig = []
df = []
cols = [['t [ps]','A(head)-B(stand)','A(head)-C(stand)','A(head)-D(stand)'],
        ['t [ps]','B(head)-A(stand)','B(head)-C(stand)','B(head)-D(stand)'],
        ['t [ps]','C(head)-A(stand)','C(head)-B(stand)','C(head)-D(stand)'],
        ['t [ps]','D(head)-A(stand)','D(head)-B(stand)','D(head)-C(stand)'],]

for i in range(0,4):
    rdf_timedev = (1e-2)*(np.loadtxt(spa_fold['rdf_fh_stand'] + '/rdf_fh_stand_timedev_ref_'+str(i+1)+'.csv', delimiter = ';', usecols=(0,1,2,3)))
    temp = (pd.DataFrame(rdf_timedev, columns = cols[i])).round(4)
    temp = temp.set_index('t [ps]')
    df.append(temp[(np.arange(len(temp)) % 100 == 0)].copy())

df_total = df[0].copy()
for i in range(0,4):
    a = df[i]
    for j in a.columns:
        if not j in df_total.columns:
            
            df_total = df_total.join(a[j])
        else:
            pass

df_total = df_total.drop(df_total.index[:TREND_CUTOFF])

# fig = plot_rdf(df_total, 'no-PBC RDF of functional heads/stands (upper layer)')
fig = plot_timedev_obs(df_total, 'no-PBC RDF of functional heads/stands (upper layer)', color_scheme, obs = 'rdf_type', layout = timedev_layout_theme)
display(fig)

# Get the closest distance between functional group and stand
sorted_df = df_total.describe().drop('count').sort_values('mean', axis = 1)
_name = str(sorted_df.columns[0])

display(format_pandas_table(sorted_df, 'General statistics for RDF(fh-stand)', caption_style, vmin=None, vmax=None, color=False,axis=0))

# Start building DataFrame to plot
min_rdf_df = pd.DataFrame(df_total[_name])

# Save data temporal evolution
save_for_origin(df_total, PROPERTY_NAME+'_timedev')

# save histograms
hist_df= calculate_histograms(df_total)
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save general statistics
save_for_origin(sorted_df, PROPERTY_NAME+'_stat')

### *Functional head - functional head distances*

In [ ]:
PROPERTY_NAME = 'ABCD_fh_fh_rdf'
fig = []
df = []
cols = [['t [ps]','A-B','A-C','A-D'],
        ['t [ps]','A-B','B-C','B-D'],
        ['t [ps]','A-C','B-C','C-D'],
        ['t [ps]','A-D','B-D','C-D'],] # Useless

for i in range(0,4):
    rdf_timedev = (1e-2)*(np.loadtxt(spa_fold['rdf_fh_fh'] + '/rdf_fh_fh_timedev_ref_'+str(i+1)+'.csv', delimiter = ';', usecols=(0,1,2,3)))
    temp = (pd.DataFrame(rdf_timedev, columns = cols[i])).round(4)
    temp = temp.set_index('t [ps]')
    df.append(temp[(np.arange(len(temp)) % 100 == 0)].copy())

df_total = df[0].copy()
for i in range(0,4):
    a = df[i]
    for j in a.columns:
        if not j in df_total.columns:
            
            df_total = df_total.join(a[j])
        else:
            pass

df_total = df_total.drop(df_total.index[:TREND_CUTOFF])

fig = plot_timedev_obs(df_total, 'no-PBC RDF of functional heads/functional heads (upper layer)', color_scheme, obs = 'rdf_type', layout = timedev_layout_theme)
display(fig)

df_dict['rdf_fh_fh_ul'] = df_total

# Perform general statistics
sorted_df = df_total.describe().drop('count').sort_values('mean', axis = 1)
display(format_pandas_table(sorted_df, 'General statistics for RDF(fh-stand)', caption_style, vmin=None, vmax=None, color=False,axis=0))

# Save data temporal evolution
save_for_origin(df_total, PROPERTY_NAME+'_timedev')

# save histograms
hist_df= calculate_histograms(df_total)
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save general statistics
save_for_origin(sorted_df, PROPERTY_NAME+'_stat')

### *Distance comparison*

In [ ]:
PROPERTY_NAME = 'ABCD_minimal_fh_stand_distance'

# Make the minimal_fh_stand_distance dataFrame
for i in df_total.columns:
    if _name[0] in i:

        extracted_col = df_total[i]
        if not i in min_rdf_df.columns:
            min_rdf_df = min_rdf_df.join(extracted_col)
        
    else:
        pass

fig = plot_minimal_distance(min_rdf_df, PROPERTY_NAME, color_scheme, layout = timedev_layout_theme)
display(fig)

# Save data temporal evolution
save_for_origin(min_rdf_df, PROPERTY_NAME+'_timedev')

# save histograms
hist_df= calculate_histograms(min_rdf_df)
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save general statistics
sorted_df = min_rdf_df.describe().drop('count').sort_values('mean', axis = 1)
save_for_origin(sorted_df, PROPERTY_NAME+'_stat')

display(sorted_df)

<a name="lower_layer"></a>
## **Lower layer interactions**
***
### *Functional head - triptycene stand distances*

In [ ]:
PROPERTY_NAME = 'EFGH_fh_stand_rdf'
fig = []
df = []
cols = [['t [ps]','E(head)-F(stand)','E(head)-G(stand)','E(head)-H(stand)'],
        ['t [ps]','F(head)-E(stand)','F(head)-G(stand)','F(head)-H(stand)'],
        ['t [ps]','G(head)-E(stand)','G(head)-F(stand)','G(head)-H(stand)'],
        ['t [ps]','H(head)-E(stand)','H(head)-F(stand)','H(head)-G(stand)'],]

for i in range(0,4):
    rdf_timedev = (1e-2)*(np.loadtxt(spa_fold['rdf_fh_stand'] + '/rdf_fh_stand_timedev_ref_'+str(i+5)+'.csv', delimiter = ';', usecols=(0,5,6,7)))
    temp = (pd.DataFrame(rdf_timedev, columns = cols[i])).round(4)
    temp = temp.set_index('t [ps]')
    df.append(temp[(np.arange(len(temp)) % 100 == 0)].copy())

df_total = df[0].copy()
for i in range(0,4):
    a = df[i]
    for j in a.columns:
        if not j in df_total.columns:
            
            df_total = df_total.join(a[j])
            
        else:
            pass

df_total = df_total.drop(df_total.index[:TREND_CUTOFF])

fig = plot_timedev_obs(df_total, 'no-PBC RDF of functional heads/stands (lower layer)', color_scheme, obs = 'rdf_type', layout = timedev_layout_theme)
display(fig)

# Get the closest distance between functional group and stand
sorted_df = df_total.describe().drop('count').sort_values('mean', axis = 1)
_name = str(sorted_df.columns[0])

display(format_pandas_table(sorted_df, 'General statistics for RDF(fh-stand)', caption_style, vmin=None, vmax=None, color=False,axis=0))

# Start building DataFrame to plot
min_rdf_df = pd.DataFrame(df_total[_name])

# Save data temporal evolution
save_for_origin(df_total, PROPERTY_NAME+'_timedev')

# save histograms
hist_df= calculate_histograms(df_total)
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save general statistics
save_for_origin(sorted_df, PROPERTY_NAME+'_stat')

### *Functional head - functional head distances*

In [ ]:
PROPERTY_NAME = 'EFGH_fh_fh_rdf'

fig = []
df = []
cols = [['t [ps]','E-F','E-G','E-H'],
        ['t [ps]','E-F','F-G','F-H'],
        ['t [ps]','E-G','F-G','G-H'],
        ['t [ps]','E-H','F-H','G-H'],] # Useless

for i in range(0,4):
    rdf_timedev = (1e-2)*(np.loadtxt(spa_fold['rdf_fh_fh'] + '/rdf_fh_fh_timedev_ref_'+str(i+5)+'.csv', delimiter = ';', usecols=(0,5,6,7)))
    temp = (pd.DataFrame(rdf_timedev, columns = cols[i])).round(4)
    temp = temp.set_index('t [ps]')
    df.append(temp[(np.arange(len(temp)) % 100 == 0)].copy())

df_total = df[0].copy()
for i in range(0,4):
    a = df[i]
    for j in a.columns:
        if not j in df_total.columns:
            
            df_total = df_total.join(a[j])
        else:
            pass
    
df_total = df_total.drop(df_total.index[:TREND_CUTOFF])

fig = plot_timedev_obs(df_total, 'no-PBC RDF of functional heads/functional heads (lower layer)', color_scheme, obs = 'rdf_type', layout = timedev_layout_theme)
display(fig)

# Perform general statistics
sorted_df = df_total.describe().drop('count').sort_values('mean', axis = 1)
display(format_pandas_table(sorted_df, 'General statistics for RDF(fh-stand)', caption_style, vmin=None, vmax=None, color=False,axis=0))

# Save data temporal evolution
save_for_origin(df_total, PROPERTY_NAME+'_timedev')

# save histograms
hist_df= calculate_histograms(df_total)
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save general statistics
save_for_origin(sorted_df, PROPERTY_NAME+'_stat')

### *Distance comparison*

In [ ]:
PROPERTY_NAME = 'EFGH_minimal_fh_stand_distance'

# Make the minimal_fh_stand_distance dataFrame
for i in df_total.columns:
    if _name[0] in i:

        extracted_col = df_total[i]
        if not i in min_rdf_df.columns:
            min_rdf_df = min_rdf_df.join(extracted_col)
        
    else:
        pass

fig = plot_minimal_distance(min_rdf_df, PROPERTY_NAME, color_scheme, layout = timedev_layout_theme)
display(fig)

# Save data temporal evolution
save_for_origin(min_rdf_df, PROPERTY_NAME+'_timedev')

# save histograms
hist_df= calculate_histograms(min_rdf_df)
save_for_origin(hist_df, PROPERTY_NAME+'_hist')

# Save general statistics
sorted_df = min_rdf_df.describe().drop('count').sort_values('mean', axis = 1)
save_for_origin(sorted_df, PROPERTY_NAME+'_stat')

display(sorted_df)

<a name="sdf"></a>
### *Spatial distribution density*
***

In [ ]:
Video("/home1/doroteja/programs/jupyter-notebooks/spa_test/sdf_video.mp4", embed = True)

<a name="cdf"></a>
## **Combined distribution functions**
***
### *Upper layer: functional head - triptycene stand distance vs functional head bend*

In [ ]:
PROPERTY_NAME = 'ABCD_cdf_rdf_adf'

cdf_adf_csv = define_fold_or_file(spa_fold['ul_cdf_adf'], file_name = 'ul_cdf_adf.csv')
cdf_adf_df = pd.read_csv(cdf_adf_csv, sep = ';')

cdf_adf_df2 = cdf_adf_df.copy()
cdf_adf_df2.rename(columns = {cdf_adf_df.columns[0]: 'd(fh-fh) [A]', cdf_adf_df.columns[1] : 'bend [deg]'}, inplace = True)
cdf_adf_df2.loc[:,'d(fh-fh) [A]'] = cdf_adf_df2.loc[:,'d(fh-fh) [A]'] * 0.01

cdf_adf_corr = define_fold_or_file(spa_fold['ul_cdf_adf'], file_name = 'ul_cdf_adf_correlation.csv')
cdf_adf_corr_df = pd.read_csv(cdf_adf_corr, sep = ';')

cdf_adf_corr_df2 = cdf_adf_corr_df.copy()
cdf_adf_corr_df2.rename(columns = {cdf_adf_corr_df.columns[0]: 'd(fh-fh) [A]', cdf_adf_corr_df.columns[1] : 'bend [deg]'}, inplace = True)
cdf_adf_corr_df2.loc[:,'d(fh-fh) [A]'] = cdf_adf_corr_df2.loc[:,'d(fh-fh) [A]'] * 0.01

fig = plot_cdf_type_data(cdf_adf_df2, cdf_adf_corr_df2, title = 'RDF (func. head/ func. head) vs ADF (bend)')

fig.update_yaxes(title_text= "<i>angle</i> [deg]", range=[0, 65], tick0=0, row=1, col=1)
fig.update_xaxes(title_text='<i>r</i> [A]', range=[4.5, 7.08], tick0=3, row=1, col=1)

fig.update_yaxes(title_text = "<i>angle</i> [deg]", range=[0, 65], tick0=0, row=1, col=2)
fig.update_xaxes(title_text='<i>r</i> [A]', range=[4.5, 7.08], tick0=3, row=1, col=2)

display(fig)

save_for_origin(cdf_adf_df2, PROPERTY_NAME + '_2d_hist')
save_for_origin(cdf_adf_corr_df2, PROPERTY_NAME + '_2d_corr_hist')

### *Functional head - functional head distance vs functional head triptycene stand distance*

In [ ]:
PROPERTY_NAME = 'ABCD_cdf_rdf_rdf'

cdf_rdf_csv = define_fold_or_file(spa_fold['ul_cdf_rdf'], file_name = 'ul_cdf_rdf.csv')
cdf_rdf_df = pd.read_csv(cdf_rdf_csv, sep = ';')

cdf_rdf_df2 = cdf_rdf_df.copy()
cdf_rdf_df2.rename(columns = {cdf_rdf_df.columns[0] : 'd(fh-fh) [A]', cdf_rdf_df.columns[1] : 'd(fh-stand) [A]'}, inplace = True)
cdf_rdf_df2.loc[:,'d(fh-fh) [A]'] = cdf_rdf_df2.loc[:,'d(fh-fh) [A]'] * 0.01
cdf_rdf_df2.loc[:,'d(fh-stand) [A]'] = cdf_rdf_df2.loc[:,'d(fh-stand) [A]'] * 0.01

cdf_rdf_corr = define_fold_or_file(spa_fold['ul_cdf_rdf'], file_name = 'ul_cdf_rdf_correlation.csv')
cdf_rdf_corr_df = pd.read_csv(cdf_rdf_corr, sep = ';')

cdf_rdf_corr_df2 = cdf_rdf_corr_df.copy()
cdf_rdf_corr_df2.rename(columns = {cdf_rdf_corr_df.columns[0]: 'd(fh-fh) [A]', cdf_rdf_corr_df.columns[1] : 'd(fh-stand) [A]'}, inplace = True)
cdf_rdf_corr_df2.loc[:,'d(fh-fh) [A]'] = cdf_rdf_corr_df2.loc[:,'d(fh-fh) [A]'] * 0.01
cdf_rdf_corr_df2.loc[:,'d(fh-stand) [A]'] = cdf_rdf_corr_df2.loc[:,'d(fh-stand) [A]'] * 0.01

fig = plot_cdf_type_data(cdf_rdf_df2, cdf_rdf_corr_df2, title = 'RDF (func. head/ func. head) vs RDF (func. head/ stand)')

fig.update_yaxes(title_text= "<i>r</i> (stand-head) [A]", range=[4, 7.08], tick0=4, row=1, col=1)
fig.update_xaxes(title_text='<i>r</i> (head-head) [A]', range=[4.5, 7.08], tick0=4, row=1, col=1)

fig.update_yaxes(title_text = "<i>r</i> (stand-head) [A]", range=[4, 7.08], tick0=4, row=1, col=2)
fig.update_xaxes(title_text='<i>r</i> (head-head) [A]', range=[4.5, 7.08], tick0=4, row=1, col=2)

display(fig)

save_for_origin(cdf_rdf_df2, PROPERTY_NAME + '_2d_hist')
save_for_origin(cdf_rdf_corr_df2, PROPERTY_NAME + '_2d_corr_hist')

### *Lower layer: functional head - triptycene stand distance vs functional head bend*

In [ ]:
PROPERTY_NAME = 'EFGH_cdf_rdf_adf'

cdf_adf_csv = define_fold_or_file(spa_fold['ll_cdf_adf'], file_name = 'll_cdf_adf.csv')
cdf_adf_df = pd.read_csv(cdf_adf_csv, sep = ';')

cdf_adf_df2 = cdf_adf_df.copy()
cdf_adf_df2.rename(columns = {cdf_adf_df.columns[0]: 'd(fh-fh) [A]', cdf_adf_df.columns[1] : 'bend [deg]'}, inplace = True)
cdf_adf_df2.loc[:,'d(fh-fh) [A]'] = cdf_adf_df2.loc[:,'d(fh-fh) [A]'] * 0.01

cdf_adf_corr = define_fold_or_file(spa_fold['ll_cdf_adf'], file_name = 'll_cdf_adf_correlation.csv')
cdf_adf_corr_df = pd.read_csv(cdf_adf_corr, sep = ';')

cdf_adf_corr_df2 = cdf_adf_corr_df.copy()
cdf_adf_corr_df2.rename(columns = {cdf_adf_corr_df.columns[0]: 'd(fh-fh) [A]', cdf_adf_corr_df.columns[1] : 'bend [deg]'}, inplace = True)
cdf_adf_corr_df2.loc[:,'d(fh-fh) [A]'] = cdf_adf_corr_df2.loc[:,'d(fh-fh) [A]'] * 0.01

fig = plot_cdf_type_data(cdf_adf_df2, cdf_adf_corr_df2, title = 'RDF (func. head/ func. head) vs ADF (bend)')

fig.update_yaxes(title_text= "<i>angle</i> [deg]", range=[0, 65], tick0=0, row=1, col=1)
fig.update_xaxes(title_text='<i>r</i> [A]', range=[4.5, 7.08], tick0=3, row=1, col=1)

fig.update_yaxes(title_text = "<i>angle</i> [deg]", range=[0, 65], tick0=0, row=1, col=2)
fig.update_xaxes(title_text='<i>r</i> [A]', range=[4.5, 7.08], tick0=3, row=1, col=2)

display(fig)

save_for_origin(cdf_adf_df2, PROPERTY_NAME + '_2d_hist')
save_for_origin(cdf_adf_corr_df2, PROPERTY_NAME + '_2d_corr_hist')

### *Functional head - functional head distance vs functional head triptycene stand distance*

In [ ]:
PROPERTY_NAME = 'EFGH_cdf_rdf_rdf'

cdf_rdf_csv = define_fold_or_file(spa_fold['ll_cdf_rdf'], file_name = 'll_cdf_rdf.csv')
cdf_rdf_df = pd.read_csv(cdf_rdf_csv, sep = ';')

cdf_rdf_df2 = cdf_rdf_df.copy()
cdf_rdf_df2.rename(columns = {cdf_rdf_df.columns[0] : 'd(fh-fh) [A]', cdf_rdf_df.columns[1] : 'd(fh-stand) [A]'}, inplace = True)
cdf_rdf_df2.loc[:,'d(fh-fh) [A]'] = cdf_rdf_df2.loc[:,'d(fh-fh) [A]'] * 0.01
cdf_rdf_df2.loc[:,'d(fh-stand) [A]'] = cdf_rdf_df2.loc[:,'d(fh-stand) [A]'] * 0.01

cdf_rdf_corr = define_fold_or_file(spa_fold['ll_cdf_rdf'], file_name = 'll_cdf_rdf_correlation.csv')
cdf_rdf_corr_df = pd.read_csv(cdf_rdf_corr, sep = ';')

cdf_rdf_corr_df2 = cdf_rdf_corr_df.copy()
cdf_rdf_corr_df2.rename(columns = {cdf_rdf_corr_df.columns[0]: 'd(fh-fh) [A]', cdf_rdf_corr_df.columns[1] : 'd(fh-stand) [A]'}, inplace = True)
cdf_rdf_corr_df2.loc[:,'d(fh-fh) [A]'] = cdf_rdf_corr_df2.loc[:,'d(fh-fh) [A]'] * 0.01
cdf_rdf_corr_df2.loc[:,'d(fh-stand) [A]'] = cdf_rdf_corr_df2.loc[:,'d(fh-stand) [A]'] * 0.01

fig = plot_cdf_type_data(cdf_rdf_df2, cdf_rdf_corr_df2, title = 'RDF (func. head/ func. head) vs RDF (func. head/ stand)')

fig.update_yaxes(title_text= "<i>r</i> (stand-head) [A]", range=[4, 7.08], tick0=4, row=1, col=1)
fig.update_xaxes(title_text='<i>r</i> (head-head) [A]', range=[4.5, 7.08], tick0=4, row=1, col=1)

fig.update_yaxes(title_text = "<i>r</i> (stand-head) [A]", range=[4, 7.08], tick0=4, row=1, col=2)
fig.update_xaxes(title_text='<i>r</i> (head-head) [A]', range=[4.5, 7.08], tick0=4, row=1, col=2)

display(fig)

save_for_origin(cdf_rdf_df2, PROPERTY_NAME + '_2d_hist')
save_for_origin(cdf_rdf_corr_df2, PROPERTY_NAME + '_2d_corr_hist')